In [ ]:
###############
## Libraries ##
###############

import sys,os
import datetime as dt
import itertools as itt

import numpy  as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot   as plt
import matplotlib.gridspec as mplg
import matplotlib.animation as mpla
import cartopy.crs as ccrs

import SDFC as sd
import scipy.stats as sc
import scipy.interpolate as sci
import statsmodels.api as sm
from multiprocessing import Pool, freeze_support

from statsmodels.gam.api import GLMGam, BSplines

path = "./data/"

### Matplotlib
plt.rcParams['text.usetex'] = True
plt.rcParams['xtick.top'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['xtick.major.width'] = 0.5
plt.rcParams['ytick.major.width'] = 0.5


#############
## Classes ##
#############

from class_gev import GEVFixedBoundLinearLink , GEVFixedBoundLinearLink_alt , GEVFixedBoundStationaryLink , GEVNonStationaryLogExp 
from class_gev import GEVNonStationaryLogExp_loc , GEVNonStationaryLogExp_scale , GEVNonStationaryLogExp_shape

## Load land-sea filter
sea_filter = xr.load_dataarray( path + "sftlf_fx_IPSL-CM6A-LR_historical_r1i1p1f1_gr.nc").sel(lat = slice(30,80))
sea_filter.coords['lon'] = (sea_filter.coords['lon'] + 180) % 360 - 180
sea_filter = sea_filter.sortby(sea_filter['lon'])

In [ ]:
## GMST and RMSTs

# tas_global
tas_global_1 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r1_1850-2014.nc")['tas'].squeeze() - 273.15
tas_global_2 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r1_2015-2029.nc")['tas'].squeeze() - 273.15
tas_global_3 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r1_2030-2059.nc")['tas'].squeeze() - 273.15
tas_global = xr.concat([tas_global_1,tas_global_2,tas_global_3],dim='time')
tas_global = tas_global.assign_coords( time = tas_global.time.dt.year.values, ensemble = 1 )
for i in range(3,33):
    temp_1 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_1850-2014.nc")['tas'].squeeze() - 273.15
    temp_2 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2015-2029.nc")['tas'].squeeze() - 273.15
    temp_3 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2030-2059.nc")['tas'].squeeze() - 273.15
    temp = xr.concat([temp_1,temp_2,temp_3],dim='time')
    temp = temp.assign_coords( time = temp.time.dt.year.values, ensemble = i-1 )
    tas_global = xr.concat([tas_global, temp], dim='ensemble')

    # tas_mean
tas_mean_1 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r1_1850-2014.nc")['tas'].squeeze() - 273.15
tas_mean_2 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r1_2015-2029.nc")['tas'].squeeze() - 273.15
tas_mean_3 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r1_2030-2059.nc")['tas'].squeeze() - 273.15
tas_mean = xr.concat([tas_mean_1,tas_mean_2,tas_mean_3],dim='time')
tas_mean = tas_mean.assign_coords( time = tas_mean.time.dt.year.values, ensemble = 1 )
for i in range(3,33):
    temp_1 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_1850-2014.nc")['tas'].squeeze() - 273.15
    temp_2 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2015-2029.nc")['tas'].squeeze() - 273.15
    temp_3 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2030-2059.nc")['tas'].squeeze() - 273.15
    temp = xr.concat([temp_1,temp_2,temp_3],dim='time')
    temp = temp.assign_coords( time = temp.time.dt.year.values, ensemble = i-1 )
    tas_mean = xr.concat([tas_mean, temp], dim='ensemble')
    
GMST = tas_global.mean(dim='ensemble')

RMST_NA = tas_mean.sel(lon=slice(-180,-30))
weights = np.cos(np.deg2rad(RMST_NA.lat))
RMST_NA = RMST_NA.weighted(weights).mean(("lon","lat"))

RMST_E = tas_mean.sel(lon=slice(-30,50))
weights = np.cos(np.deg2rad(RMST_E.lat))
RMST_E = RMST_E.weighted(weights).mean(("lon","lat"))

RMST_A = tas_mean.sel(lon=slice(50,180))
weights = np.cos(np.deg2rad(RMST_A.lat))
RMST_A = RMST_A.weighted(weights).mean(("lon","lat"))
    
# Plot

fig = plt.figure(figsize=(7,4), dpi=1200)

# GMST
ax = plt.subplot(2,2,1)

for i in range(1,tas_global['ensemble'].size):
    data = tas_global.sel(ensemble=i)
    if i == 1:
        ax.plot(data.time, data.values, color='red', alpha=0.1, label="Members")
    else:
        ax.plot(data.time, data.values, color='red', alpha=0.1)
ax.plot(tas_global.time, GMST, color='black', label="Ensemble mean")

ax.set_ylabel("GMST [°C]")
ax.set_xticklabels("")
ax.set_xlim(1850,2059)
ax.text(0.06, 0.97, 'a', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.grid(linestyle='dotted', linewidth=0.3)
ax.legend(fontsize=10, loc='upper center')


# RMST_NA
ax = plt.subplot(2,2,2)

for i in range(1,RMST_NA['ensemble'].size):
    data = RMST_NA.sel(ensemble=i)
    if i == 1:
        ax.plot(data.time, data.values, color='red', alpha=0.1, label="Members")
    else:
        ax.plot(data.time, data.values, color='red', alpha=0.1)
ax.plot(tas_global.time, RMST_NA.mean(dim='ensemble'), color='black')

ax.set_ylabel("RMST NA [°C]")
ax.set_xticklabels("")
ax.set_xlim(1850,2059)
ax.text(0.06, 0.97, 'b', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.grid(linestyle='dotted', linewidth=0.3)


# RMST_E
ax = plt.subplot(2,2,3)

for i in range(1,RMST_E['ensemble'].size):
    data = RMST_E.sel(ensemble=i)
    if i == 1:
        ax.plot(data.time, data.values, color='red', alpha=0.1, label="Members")
    else:
        ax.plot(data.time, data.values, color='red', alpha=0.1)
ax.plot(tas_global.time, RMST_E.mean(dim='ensemble'), color='black')

ax.set_ylabel("RMST E [°C]")
ax.set_xlabel("Time")
ax.set_xlim(1850,2059)
ax.text(0.06, 0.97, 'c', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.grid(linestyle='dotted', linewidth=0.3)


# RMST_A
ax = plt.subplot(2,2,4)

for i in range(1,RMST_A['ensemble'].size):
    data = RMST_A.sel(ensemble=i)
    if i == 1:
        ax.plot(data.time, data.values, color='red', alpha=0.1, label="Members")
    else:
        ax.plot(data.time, data.values, color='red', alpha=0.1)
ax.plot(tas_global.time, RMST_A.mean(dim='ensemble'), color='black')

ax.set_ylabel("RMST A [°C]")
ax.set_xlabel("Time")
ax.set_xlim(1850,2059)
ax.text(0.06, 0.97, 'd', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.grid(linestyle='dotted', linewidth=0.3)

        
plt.tight_layout()
plt.show()


In [ ]:
## Maps model M1,M2 with GMST

year_limit = 2014

for n_years in [70, 100, 150]:

    ## Load data
    proba_above_bound = xr.load_dataarray(path + "proba_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")
    proba_median_above_bound = xr.load_dataarray(path + "proba_median_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")
    return_time_100_median = xr.load_dataarray(path + "return_time_100_median_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")

    lon = proba_above_bound['lon']
    lat = proba_above_bound['lat'] 

    ## Plot    
    xticks = [0, 50, 100, 150, -50, -100, -150]
    yticks = [35, 45, 55, 65,75]
    xticklabels = ['0°', '50°E', '100°E', '150°E', '50°W', '100°W', '150°W']
    yticklabels = ['35°N', '45°N', '55°N', '65°N', '75°N']
    letters_tab = [['a','c','e'],['b','d','f']]


    fig = plt.figure(figsize=(7,5.5), dpi=1200)
    gs = fig.add_gridspec(3,2)

    for idm,m in enumerate(["m1","m2"]):

        ## Proba above
        cmap = mpl.cm.get_cmap('Reds', 10)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = proba_above_bound.sel( model = m ).values*100
        tab[tab == 0] = -100
        tab[sea_filter == 0] = np.nan
        ax = fig.add_subplot(gs[0,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', vmin=0, vmax=100, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][0], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
        if m =="m1":
            ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1$)", fontsize=10)
        elif m =="m2":
            ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2$)", fontsize=10)

        cbar_ax = fig.add_axes([0.285, 0.75, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Probability to exceed the upper bound at least once [\%]", spacing='proportional', cax=cbar_ax, orientation='horizontal')


        ## Return time median bound
        cmap = mpl.cm.get_cmap('Reds_r', 11)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['white'])
        cmap.set_over(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = 1 / proba_median_above_bound.sel( model = m ).values
        tab[proba_above_bound.sel( model = m ).values == 0] = -100
        tab[tab == np.inf] = 1e50
        tab[np.isnan(tab)] = 1e50
        tab[sea_filter == 0] = np.nan
        tab[proba_above_bound.sel( model = m ).values == 0] = np.nan
        ax = fig.add_subplot(gs[1,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=mpl.colors.LogNorm(vmin=1,vmax=6e3), aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][1], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.43, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Return time of the upper bound [y]", spacing='proportional', cax=cbar_ax, orientation='horizontal', extend='max')


        ## Under/over estimation of proba at 100y

        tab = return_time_100_median.sel( model = m)
        for line in tab:
            temp1 = line > 100
            temp2 = line < 100
            line[ temp1 ] = - line[ temp1 ] / 100
            line[ temp2 ] = 100 / line[ temp2]

        cmap = mpl.cm.get_cmap('RdBu_r', 13)
        norm = mpl.colors.BoundaryNorm([-5,-4,-3,-2.5,-2,-1.5,-1,1,1.5,2,2.5,3,4,5], cmap.N)
        ax = fig.add_subplot(gs[2,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels(xticklabels, fontsize=8)
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][2], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.08, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Ratio between the GEV and the empirical probabilities of a centennial event", cax=cbar_ax, orientation='horizontal', extend='both')
        cbar.set_ticks([-5, -4, -3, -2.5, -2, -1.5, -1, 1, 1.5, 2, 2.5, 3, 4, 5])
        cbar.set_ticklabels(["-5", "-4", "-3", "-2.5", "-2", "-1.5", "-1", "1", "1.5", "2", "2.5", "3", "4", "5"])

    fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.1 , top = 1 , hspace = 0.25 , wspace = 0.08 )
    plt.show()

n_years = 70
for year_limit in [2025, 2000, 1975]:
    
    ## Load data
    proba_above_bound = xr.load_dataarray(path + "proba_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")
    proba_median_above_bound = xr.load_dataarray(path + "proba_median_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")
    return_time_100_median = xr.load_dataarray(path + "return_time_100_median_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")

    lon = proba_above_bound['lon']
    lat = proba_above_bound['lat'] 

    ## Plot    
    xticks = [0, 50, 100, 150, -50, -100, -150]
    yticks = [35, 45, 55, 65,75]
    xticklabels = ['0°', '50°E', '100°E', '150°E', '50°W', '100°W', '150°W']
    yticklabels = ['35°N', '45°N', '55°N', '65°N', '75°N']
    letters_tab = [['a','c','e'],['b','d','f']]


    fig = plt.figure(figsize=(7,5.5), dpi=1200)
    gs = fig.add_gridspec(3,2)

    for idm,m in enumerate(["m1","m2"]):

        ## Proba above
        cmap = mpl.cm.get_cmap('Reds', 10)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = proba_above_bound.sel( model = m ).values*100
        tab[tab == 0] = -100
        tab[sea_filter == 0] = np.nan
        ax = fig.add_subplot(gs[0,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', vmin=0, vmax=100, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][0], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
        if m =="m1":
            ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1$)", fontsize=10)
        elif m =="m2":
            ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2$)", fontsize=10)

        cbar_ax = fig.add_axes([0.285, 0.75, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Probability to exceed the upper bound at least once [\%]", spacing='proportional', cax=cbar_ax, orientation='horizontal')


        ## Return time median bound
        cmap = mpl.cm.get_cmap('Reds_r', 11)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['white'])
        cmap.set_over(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = 1 / proba_median_above_bound.sel( model = m ).values
        tab[proba_above_bound.sel( model = m ).values == 0] = -100
        tab[tab == np.inf] = 1e50
        tab[np.isnan(tab)] = 1e50
        tab[sea_filter == 0] = np.nan
        tab[proba_above_bound.sel( model = m ).values == 0] = np.nan
        ax = fig.add_subplot(gs[1,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=mpl.colors.LogNorm(vmin=1,vmax=6e3), aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][1], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.43, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Return time of the upper bound [y]", spacing='proportional', cax=cbar_ax, orientation='horizontal', extend='max')


        ## Under/over estimation of proba at 100y

        tab = return_time_100_median.sel( model = m)
        for line in tab:
            temp1 = line > 100
            temp2 = line < 100
            line[ temp1 ] = - line[ temp1 ] / 100
            line[ temp2 ] = 100 / line[ temp2]

        cmap = mpl.cm.get_cmap('RdBu_r', 13)
        norm = mpl.colors.BoundaryNorm([-5,-4,-3,-2.5,-2,-1.5,-1,1,1.5,2,2.5,3,4,5], cmap.N)
        ax = fig.add_subplot(gs[2,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels(xticklabels, fontsize=8)
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][2], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.08, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Ratio between the GEV and the empirical probabilities of a centennial event", cax=cbar_ax, orientation='horizontal', extend='both')
        cbar.set_ticks([-5, -4, -3, -2.5, -2, -1.5, -1, 1, 1.5, 2, 2.5, 3, 4, 5])
        cbar.set_ticklabels(["-5", "-4", "-3", "-2.5", "-2", "-1.5", "-1", "1", "1.5", "2", "2.5", "3", "4", "5"])

    fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.1 , top = 1 , hspace = 0.25 , wspace = 0.08 )
    plt.show()
    
    

In [ ]:
## Maps M1,M2 with RMST

year_limit = 2014

for n_years in [70, 100, 150]:

    ## Load data
    proba_above_bound = xr.load_dataarray(path + "proba_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")
    proba_median_above_bound = xr.load_dataarray(path + "proba_median_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")
    return_time_100_median = xr.load_dataarray(path + "return_time_100_median_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")

    lon = proba_above_bound['lon']
    lat = proba_above_bound['lat'] 

    ## Plot    
    xticks = [0, 50, 100, 150, -50, -100, -150]
    yticks = [35, 45, 55, 65,75]
    xticklabels = ['0°', '50°E', '100°E', '150°E', '50°W', '100°W', '150°W']
    yticklabels = ['35°N', '45°N', '55°N', '65°N', '75°N']
    letters_tab = [['a','c','e'],['b','d','f']]


    fig = plt.figure(figsize=(7,5.5), dpi=1200)
    gs = fig.add_gridspec(3,2)

    for idm,m in enumerate(["m1","m2"]):

        ## Proba above
        cmap = mpl.cm.get_cmap('Reds', 10)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = proba_above_bound.sel( model = m ).values*100
        tab[tab == 0] = -100
        tab[sea_filter == 0] = np.nan
        ax = fig.add_subplot(gs[0,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', vmin=0, vmax=100, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][0], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
        if m =="m1":
            ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1$)", fontsize=10)
        elif m =="m2":
            ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2$)", fontsize=10)

        cbar_ax = fig.add_axes([0.285, 0.75, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Probability to exceed the upper bound at least once [\%]", spacing='proportional', cax=cbar_ax, orientation='horizontal')


        ## Return time median bound
        cmap = mpl.cm.get_cmap('Reds_r', 11)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['white'])
        cmap.set_over(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = 1 / proba_median_above_bound.sel( model = m ).values
        tab[proba_above_bound.sel( model = m ).values == 0] = -100
        tab[tab == np.inf] = 1e50
        tab[np.isnan(tab)] = 1e50
        tab[sea_filter == 0] = np.nan
        tab[proba_above_bound.sel( model = m ).values == 0] = np.nan
        ax = fig.add_subplot(gs[1,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=mpl.colors.LogNorm(vmin=1,vmax=6e3), aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][1], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.43, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Return time of the upper bound [y]", spacing='proportional', cax=cbar_ax, orientation='horizontal', extend='max')


        ## Under/over estimation of proba at 100y

        tab = return_time_100_median.sel( model = m)
        for line in tab:
            temp1 = line > 100
            temp2 = line < 100
            line[ temp1 ] = - line[ temp1 ] / 100
            line[ temp2 ] = 100 / line[ temp2]

        cmap = mpl.cm.get_cmap('RdBu_r', 13)
        norm = mpl.colors.BoundaryNorm([-5,-4,-3,-2.5,-2,-1.5,-1,1,1.5,2,2.5,3,4,5], cmap.N)
        ax = fig.add_subplot(gs[2,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels(xticklabels, fontsize=8)
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][2], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.08, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Ratio between the GEV and the empirical probabilities of a centennial event", cax=cbar_ax, orientation='horizontal', extend='both')
        cbar.set_ticks([-5, -4, -3, -2.5, -2, -1.5, -1, 1, 1.5, 2, 2.5, 3, 4, 5])
        cbar.set_ticklabels(["-5", "-4", "-3", "-2.5", "-2", "-1.5", "-1", "1", "1.5", "2", "2.5", "3", "4", "5"])

    fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.1 , top = 1 , hspace = 0.25 , wspace = 0.08 )
    plt.show()

n_years = 70
for year_limit in [2025, 2000, 1975]:
    
    ## Load data
    proba_above_bound = xr.load_dataarray(path + "proba_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")
    proba_median_above_bound = xr.load_dataarray(path + "proba_median_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")
    return_time_100_median = xr.load_dataarray(path + "return_time_100_median_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")

    lon = proba_above_bound['lon']
    lat = proba_above_bound['lat'] 

    ## Plot    
    xticks = [0, 50, 100, 150, -50, -100, -150]
    yticks = [35, 45, 55, 65,75]
    xticklabels = ['0°', '50°E', '100°E', '150°E', '50°W', '100°W', '150°W']
    yticklabels = ['35°N', '45°N', '55°N', '65°N', '75°N']
    letters_tab = [['a','c','e'],['b','d','f']]


    fig = plt.figure(figsize=(7,5.5), dpi=1200)
    gs = fig.add_gridspec(3,2)

    for idm,m in enumerate(["m1","m2"]):

        ## Proba above
        cmap = mpl.cm.get_cmap('Reds', 10)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = proba_above_bound.sel( model = m ).values*100
        tab[tab == 0] = -100
        tab[sea_filter == 0] = np.nan
        ax = fig.add_subplot(gs[0,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', vmin=0, vmax=100, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][0], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
        if m =="m1":
            ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1$)", fontsize=10)
        elif m =="m2":
            ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2$)", fontsize=10)

        cbar_ax = fig.add_axes([0.285, 0.75, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Probability to exceed the upper bound at least once [\%]", spacing='proportional', cax=cbar_ax, orientation='horizontal')


        ## Return time median bound
        cmap = mpl.cm.get_cmap('Reds_r', 11)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['white'])
        cmap.set_over(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = 1 / proba_median_above_bound.sel( model = m ).values
        tab[proba_above_bound.sel( model = m ).values == 0] = -100
        tab[tab == np.inf] = 1e50
        tab[np.isnan(tab)] = 1e50
        tab[sea_filter == 0] = np.nan
        tab[proba_above_bound.sel( model = m ).values == 0] = np.nan
        ax = fig.add_subplot(gs[1,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=mpl.colors.LogNorm(vmin=1,vmax=6e3), aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][1], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.43, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Return time of the upper bound [y]", spacing='proportional', cax=cbar_ax, orientation='horizontal', extend='max')


        ## Under/over estimation of proba at 100y

        tab = return_time_100_median.sel( model = m)
        for line in tab:
            temp1 = line > 100
            temp2 = line < 100
            line[ temp1 ] = - line[ temp1 ] / 100
            line[ temp2 ] = 100 / line[ temp2]

        cmap = mpl.cm.get_cmap('RdBu_r', 13)
        norm = mpl.colors.BoundaryNorm([-5,-4,-3,-2.5,-2,-1.5,-1,1,1.5,2,2.5,3,4,5], cmap.N)
        ax = fig.add_subplot(gs[2,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels(xticklabels, fontsize=8)
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][2], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.08, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Ratio between the GEV and the empirical probabilities of a centennial event", cax=cbar_ax, orientation='horizontal', extend='both')
        cbar.set_ticks([-5, -4, -3, -2.5, -2, -1.5, -1, 1, 1.5, 2, 2.5, 3, 4, 5])
        cbar.set_ticklabels(["-5", "-4", "-3", "-2.5", "-2", "-1.5", "-1", "1", "1.5", "2", "2.5", "3", "4", "5"])

    fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.1 , top = 1 , hspace = 0.25 , wspace = 0.08 )
    plt.show()

In [ ]:
## Maps M1_B, M2_B with GMST

year_limit = 2014

for n_years in [70, 100, 150]:

    ## Load data
    proba_above_bound = xr.load_dataarray(path + "proba_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")
    proba_median_above_bound = xr.load_dataarray(path + "proba_median_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")
    return_time_100_median = xr.load_dataarray(path + "return_time_100_median_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")

    lon = proba_above_bound['lon']
    lat = proba_above_bound['lat'] 

    ## Plot    
    xticks = [0, 50, 100, 150, -50, -100, -150]
    yticks = [35, 45, 55, 65,75]
    xticklabels = ['0°', '50°E', '100°E', '150°E', '50°W', '100°W', '150°W']
    yticklabels = ['35°N', '45°N', '55°N', '65°N', '75°N']
    letters_tab = [['a','c','e'],['b','d','f']]


    fig = plt.figure(figsize=(7,5.5), dpi=1200)
    gs = fig.add_gridspec(3,2)

    for idm,m in enumerate(["m1_B","m2_B"]):

        ## Proba above
        cmap = mpl.cm.get_cmap('Reds', 10)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = proba_above_bound.sel( model = m ).values*100
        tab[tab == 0] = -100
        tab[sea_filter == 0] = np.nan
        ax = fig.add_subplot(gs[0,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', vmin=0, vmax=100, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][0], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
        if m =="m1_B":
            ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1^B$)", fontsize=10)
        elif m =="m2_B":
            ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2^B$)", fontsize=10)

        cbar_ax = fig.add_axes([0.285, 0.75, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Probability to exceed the upper bound at least once [\%]", spacing='proportional', cax=cbar_ax, orientation='horizontal')


        ## Return time median bound
        cmap = mpl.cm.get_cmap('Reds_r', 11)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['white'])
        cmap.set_over(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = 1 / proba_median_above_bound.sel( model = m ).values
        tab[proba_above_bound.sel( model = m ).values == 0] = -100
        tab[tab == np.inf] = 1e50
        tab[np.isnan(tab)] = 1e50
        tab[sea_filter == 0] = np.nan
        tab[proba_above_bound.sel( model = m ).values == 0] = np.nan
        ax = fig.add_subplot(gs[1,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=mpl.colors.LogNorm(vmin=1,vmax=6e3), aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][1], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.43, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Return time of the upper bound [y]", spacing='proportional', cax=cbar_ax, orientation='horizontal', extend='max')


        ## Under/over estimation of proba at 100y

        tab = return_time_100_median.sel( model = m)
        for line in tab:
            temp1 = line > 100
            temp2 = line < 100
            line[ temp1 ] = - line[ temp1 ] / 100
            line[ temp2 ] = 100 / line[ temp2]

        cmap = mpl.cm.get_cmap('RdBu_r', 13)
        norm = mpl.colors.BoundaryNorm([-5,-4,-3,-2.5,-2,-1.5,-1,1,1.5,2,2.5,3,4,5], cmap.N)
        ax = fig.add_subplot(gs[2,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels(xticklabels, fontsize=8)
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][2], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.08, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Ratio between the GEV and the empirical probabilities of a centennial event", cax=cbar_ax, orientation='horizontal', extend='both')
        cbar.set_ticks([-5, -4, -3, -2.5, -2, -1.5, -1, 1, 1.5, 2, 2.5, 3, 4, 5])
        cbar.set_ticklabels(["-5", "-4", "-3", "-2.5", "-2", "-1.5", "-1", "1", "1.5", "2", "2.5", "3", "4", "5"])

    fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.1 , top = 1 , hspace = 0.25 , wspace = 0.08 )
    plt.show()

n_years = 70
for year_limit in [2025, 2000, 1975]:
    
    ## Load data
    proba_above_bound = xr.load_dataarray(path + "proba_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")
    proba_median_above_bound = xr.load_dataarray(path + "proba_median_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")
    return_time_100_median = xr.load_dataarray(path + "return_time_100_median_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3.nc")

    lon = proba_above_bound['lon']
    lat = proba_above_bound['lat'] 

    ## Plot    
    xticks = [0, 50, 100, 150, -50, -100, -150]
    yticks = [35, 45, 55, 65,75]
    xticklabels = ['0°', '50°E', '100°E', '150°E', '50°W', '100°W', '150°W']
    yticklabels = ['35°N', '45°N', '55°N', '65°N', '75°N']
    letters_tab = [['a','c','e'],['b','d','f']]


    fig = plt.figure(figsize=(7,5.5), dpi=1200)
    gs = fig.add_gridspec(3,2)

    for idm,m in enumerate(["m1_B","m2_B"]):

        ## Proba above
        cmap = mpl.cm.get_cmap('Reds', 10)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = proba_above_bound.sel( model = m ).values*100
        tab[tab == 0] = -100
        tab[sea_filter == 0] = np.nan
        ax = fig.add_subplot(gs[0,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', vmin=0, vmax=100, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][0], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
        if m =="m1_B":
            ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1^B$)", fontsize=10)
        elif m =="m2_B":
            ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2^B$)", fontsize=10)

        cbar_ax = fig.add_axes([0.285, 0.75, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Probability to exceed the upper bound at least once [\%]", spacing='proportional', cax=cbar_ax, orientation='horizontal')


        ## Return time median bound
        cmap = mpl.cm.get_cmap('Reds_r', 11)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['white'])
        cmap.set_over(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = 1 / proba_median_above_bound.sel( model = m ).values
        tab[proba_above_bound.sel( model = m ).values == 0] = -100
        tab[tab == np.inf] = 1e50
        tab[np.isnan(tab)] = 1e50
        tab[sea_filter == 0] = np.nan
        tab[proba_above_bound.sel( model = m ).values == 0] = np.nan
        ax = fig.add_subplot(gs[1,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=mpl.colors.LogNorm(vmin=1,vmax=6e3), aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][1], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.43, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Return time of the upper bound [y]", spacing='proportional', cax=cbar_ax, orientation='horizontal', extend='max')


        ## Under/over estimation of proba at 100y

        tab = return_time_100_median.sel( model = m)
        for line in tab:
            temp1 = line > 100
            temp2 = line < 100
            line[ temp1 ] = - line[ temp1 ] / 100
            line[ temp2 ] = 100 / line[ temp2]

        cmap = mpl.cm.get_cmap('RdBu_r', 13)
        norm = mpl.colors.BoundaryNorm([-5,-4,-3,-2.5,-2,-1.5,-1,1,1.5,2,2.5,3,4,5], cmap.N)
        ax = fig.add_subplot(gs[2,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels(xticklabels, fontsize=8)
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][2], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.08, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Ratio between the GEV and the empirical probabilities of a centennial event", cax=cbar_ax, orientation='horizontal', extend='both')
        cbar.set_ticks([-5, -4, -3, -2.5, -2, -1.5, -1, 1, 1.5, 2, 2.5, 3, 4, 5])
        cbar.set_ticklabels(["-5", "-4", "-3", "-2.5", "-2", "-1.5", "-1", "1", "1.5", "2", "2.5", "3", "4", "5"])

    fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.1 , top = 1 , hspace = 0.25 , wspace = 0.08 )
    plt.show()

In [ ]:
## Maps M1_B,M2_B with RMST

year_limit = 2014

for n_years in [70, 100, 150]:

    ## Load data
    proba_above_bound = xr.load_dataarray(path + "proba_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")
    proba_median_above_bound = xr.load_dataarray(path + "proba_median_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")
    return_time_100_median = xr.load_dataarray(path + "return_time_100_median_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")

    lon = proba_above_bound['lon']
    lat = proba_above_bound['lat'] 

    ## Plot    
    xticks = [0, 50, 100, 150, -50, -100, -150]
    yticks = [35, 45, 55, 65,75]
    xticklabels = ['0°', '50°E', '100°E', '150°E', '50°W', '100°W', '150°W']
    yticklabels = ['35°N', '45°N', '55°N', '65°N', '75°N']
    letters_tab = [['a','c','e'],['b','d','f']]


    fig = plt.figure(figsize=(7,5.5), dpi=1200)
    gs = fig.add_gridspec(3,2)

    for idm,m in enumerate(["m1_B","m2_B"]):

        ## Proba above
        cmap = mpl.cm.get_cmap('Reds', 10)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = proba_above_bound.sel( model = m ).values*100
        tab[tab == 0] = -100
        tab[sea_filter == 0] = np.nan
        ax = fig.add_subplot(gs[0,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', vmin=0, vmax=100, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][0], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
        if m =="m1":
            ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1$)", fontsize=10)
        elif m =="m2":
            ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2$)", fontsize=10)

        cbar_ax = fig.add_axes([0.285, 0.75, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Probability to exceed the upper bound at least once [\%]", spacing='proportional', cax=cbar_ax, orientation='horizontal')


        ## Return time median bound
        cmap = mpl.cm.get_cmap('Reds_r', 11)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['white'])
        cmap.set_over(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = 1 / proba_median_above_bound.sel( model = m ).values
        tab[proba_above_bound.sel( model = m ).values == 0] = -100
        tab[tab == np.inf] = 1e50
        tab[np.isnan(tab)] = 1e50
        tab[sea_filter == 0] = np.nan
        tab[proba_above_bound.sel( model = m ).values == 0] = np.nan
        ax = fig.add_subplot(gs[1,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=mpl.colors.LogNorm(vmin=1,vmax=6e3), aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][1], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.43, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Return time of the upper bound [y]", spacing='proportional', cax=cbar_ax, orientation='horizontal', extend='max')


        ## Under/over estimation of proba at 100y

        tab = return_time_100_median.sel( model = m)
        for line in tab:
            temp1 = line > 100
            temp2 = line < 100
            line[ temp1 ] = - line[ temp1 ] / 100
            line[ temp2 ] = 100 / line[ temp2]

        cmap = mpl.cm.get_cmap('RdBu_r', 13)
        norm = mpl.colors.BoundaryNorm([-5,-4,-3,-2.5,-2,-1.5,-1,1,1.5,2,2.5,3,4,5], cmap.N)
        ax = fig.add_subplot(gs[2,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels(xticklabels, fontsize=8)
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][2], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.08, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Ratio between the GEV and the empirical probabilities of a centennial event", cax=cbar_ax, orientation='horizontal', extend='both')
        cbar.set_ticks([-5, -4, -3, -2.5, -2, -1.5, -1, 1, 1.5, 2, 2.5, 3, 4, 5])
        cbar.set_ticklabels(["-5", "-4", "-3", "-2.5", "-2", "-1.5", "-1", "1", "1.5", "2", "2.5", "3", "4", "5"])

    fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.1 , top = 1 , hspace = 0.25 , wspace = 0.08 )
    plt.show()

n_years = 70
for year_limit in [2025, 2000, 1975]:
    
    ## Load data
    proba_above_bound = xr.load_dataarray(path + "proba_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")
    proba_median_above_bound = xr.load_dataarray(path + "proba_median_above_bound_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")
    return_time_100_median = xr.load_dataarray(path + "return_time_100_median_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_regcov.nc")

    lon = proba_above_bound['lon']
    lat = proba_above_bound['lat'] 

    ## Plot    
    xticks = [0, 50, 100, 150, -50, -100, -150]
    yticks = [35, 45, 55, 65,75]
    xticklabels = ['0°', '50°E', '100°E', '150°E', '50°W', '100°W', '150°W']
    yticklabels = ['35°N', '45°N', '55°N', '65°N', '75°N']
    letters_tab = [['a','c','e'],['b','d','f']]


    fig = plt.figure(figsize=(7,5.5), dpi=1200)
    gs = fig.add_gridspec(3,2)

    for idm,m in enumerate(["m1_B","m2_B"]):

        ## Proba above
        cmap = mpl.cm.get_cmap('Reds', 10)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = proba_above_bound.sel( model = m ).values*100
        tab[tab == 0] = -100
        tab[sea_filter == 0] = np.nan
        ax = fig.add_subplot(gs[0,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', vmin=0, vmax=100, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][0], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
        if m =="m1":
            ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1$)", fontsize=10)
        elif m =="m2":
            ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2$)", fontsize=10)

        cbar_ax = fig.add_axes([0.285, 0.75, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Probability to exceed the upper bound at least once [\%]", spacing='proportional', cax=cbar_ax, orientation='horizontal')


        ## Return time median bound
        cmap = mpl.cm.get_cmap('Reds_r', 11)
        cmap.set_under(color=mpl.colors.CSS4_COLORS['white'])
        cmap.set_over(color=mpl.colors.CSS4_COLORS['lightblue'])
        tab = 1 / proba_median_above_bound.sel( model = m ).values
        tab[proba_above_bound.sel( model = m ).values == 0] = -100
        tab[tab == np.inf] = 1e50
        tab[np.isnan(tab)] = 1e50
        tab[sea_filter == 0] = np.nan
        tab[proba_above_bound.sel( model = m ).values == 0] = np.nan
        ax = fig.add_subplot(gs[1,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=mpl.colors.LogNorm(vmin=1,vmax=6e3), aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels("")
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][1], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.43, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Return time of the upper bound [y]", spacing='proportional', cax=cbar_ax, orientation='horizontal', extend='max')


        ## Under/over estimation of proba at 100y

        tab = return_time_100_median.sel( model = m)
        for line in tab:
            temp1 = line > 100
            temp2 = line < 100
            line[ temp1 ] = - line[ temp1 ] / 100
            line[ temp2 ] = 100 / line[ temp2]

        cmap = mpl.cm.get_cmap('RdBu_r', 13)
        norm = mpl.colors.BoundaryNorm([-5,-4,-3,-2.5,-2,-1.5,-1,1,1.5,2,2.5,3,4,5], cmap.N)
        ax = fig.add_subplot(gs[2,idm], projection=ccrs.PlateCarree())
        cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

        ax.coastlines('50m', color='0', linewidth=.3)   
        ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
        ax.set_yticks(yticks, crs=ccrs.PlateCarree())
        ax.set_xticks(xticks, crs=ccrs.PlateCarree())
        ax.set_xticklabels(xticklabels, fontsize=8)
        if idm == 0:
            ax.set_yticklabels(yticklabels, fontsize=8)
        else:
            ax.set_yticklabels("")
        ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
        ax.text(-0.06, 1.2, letters_tab[idm][2], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

        cbar_ax = fig.add_axes([0.285, 0.08, 0.48, 0.02])
        cbar = fig.colorbar(cp, label="Ratio between the GEV and the empirical probabilities of a centennial event", cax=cbar_ax, orientation='horizontal', extend='both')
        cbar.set_ticks([-5, -4, -3, -2.5, -2, -1.5, -1, 1, 1.5, 2, 2.5, 3, 4, 5])
        cbar.set_ticklabels(["-5", "-4", "-3", "-2.5", "-2", "-1.5", "-1", "1", "1.5", "2", "2.5", "3", "4", "5"])

    fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.1 , top = 1 , hspace = 0.25 , wspace = 0.08 )
    plt.show()

In [ ]:
## Load data for grid point analysis

    # tas_global
tas_global_1 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r1_1850-2014.nc")['tas'].squeeze() - 273.15
tas_global_2 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r1_2015-2029.nc")['tas'].squeeze() - 273.15
tas_global_3 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r1_2030-2059.nc")['tas'].squeeze() - 273.15
tas_global = xr.concat([tas_global_1,tas_global_2,tas_global_3],dim='time')
tas_global = tas_global.assign_coords( time = tas_global.time.dt.year.values, ensemble = 1 )
for i in range(3,33):
    temp_1 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_1850-2014.nc")['tas'].squeeze() - 273.15
    temp_2 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2015-2029.nc")['tas'].squeeze() - 273.15
    temp_3 = xr.load_dataset(path + "tas_global_mean_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2030-2059.nc")['tas'].squeeze() - 273.15
    temp = xr.concat([temp_1,temp_2,temp_3],dim='time')
    temp = temp.assign_coords( time = temp.time.dt.year.values, ensemble = i-1 )
    tas_global = xr.concat([tas_global, temp], dim='ensemble')

    # tas_mean
tas_mean_1 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r1_1850-2014.nc")['tas'].squeeze() - 273.15
tas_mean_2 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r1_2015-2029.nc")['tas'].squeeze() - 273.15
tas_mean_3 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r1_2030-2059.nc")['tas'].squeeze() - 273.15
tas_mean = xr.concat([tas_mean_1,tas_mean_2,tas_mean_3],dim='time')
tas_mean = tas_mean.assign_coords( time = tas_mean.time.dt.year.values, ensemble = 1 )
for i in range(3,33):
    temp_1 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_1850-2014.nc")['tas'].squeeze() - 273.15
    temp_2 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2015-2029.nc")['tas'].squeeze() - 273.15
    temp_3 = xr.load_dataset(path + "tasmean_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2030-2059.nc")['tas'].squeeze() - 273.15
    temp = xr.concat([temp_1,temp_2,temp_3],dim='time')
    temp = temp.assign_coords( time = temp.time.dt.year.values, ensemble = i-1 )
    tas_mean = xr.concat([tas_mean, temp], dim='ensemble')

    # tas_max
tas_max_1 = xr.load_dataset(path + "tasmax_year_IPSL-CM6A-LR_historical-EXT_r1_1850-2014.nc")['tasmax'].squeeze() - 273.15
tas_max_2 = xr.load_dataset(path + "tasmax_year_IPSL-CM6A-LR_historical-EXT_r1_2015-2029.nc")['tasmax'].squeeze() - 273.15
tas_max_3 = xr.load_dataset(path + "tasmax_year_IPSL-CM6A-LR_historical-EXT_r1_2030-2059.nc")['tasmax'].squeeze() - 273.15
tas_max = xr.concat([tas_max_1,tas_max_2,tas_max_3],dim='time')
tas_max = tas_max.assign_coords( time = tas_max.time.dt.year.values, ensemble = 1 )
for i in range(3,33):
    temp_1 = xr.load_dataset(path + "tasmax_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_1850-2014.nc")['tasmax'].squeeze() - 273.15
    temp_2 = xr.load_dataset(path + "tasmax_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2015-2029.nc")['tasmax'].squeeze() - 273.15
    temp_3 = xr.load_dataset(path + "tasmax_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2030-2059.nc")['tasmax'].squeeze() - 273.15
    temp = xr.concat([temp_1,temp_2,temp_3],dim='time')
    temp = temp.assign_coords( time = temp.time.dt.year.values, ensemble = i-1 )
    tas_max = xr.concat([tas_max, temp], dim='ensemble')

    # t500
t500_0 = xr.load_dataset(path + "t500max_year_IPSL-CM6A-LR_historical-EXT_r1_1850-1949.nc")['ta'].squeeze() 
t500_1 = xr.load_dataset(path + "t500max_year_IPSL-CM6A-LR_historical-EXT_r1_1950-2014.nc")['ta'].squeeze() 
t500_2 = xr.load_dataset(path + "t500max_year_IPSL-CM6A-LR_historical-EXT_r1_2015-2029.nc")['ta'].squeeze() 
t500_3 = xr.load_dataset(path + "t500max_year_IPSL-CM6A-LR_historical-EXT_r1_2030-2059.nc")['ta'].squeeze()
t500 = xr.concat([t500_0,t500_1,t500_2,t500_3],dim='time')
t500 = t500.assign_coords( time = t500.time.dt.year.values, ensemble = 1 )
for i in range(3,33):
    temp_1 = xr.load_dataset(path + "t500max_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_1950-2014.nc")['ta'].squeeze() 
    temp_2 = xr.load_dataset(path + "t500max_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2015-2029.nc")['ta'].squeeze() 
    temp_3 = xr.load_dataset(path + "t500max_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2030-2059.nc")['ta'].squeeze() 
    if i in [4,5,6,7,8,9,10,11,28,29,30,31,32]:
        temp_0 = xr.load_dataset(path + "t500max_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_1850-1949.nc")['ta'].squeeze() 
        temp = xr.concat([temp_0,temp_1,temp_2,temp_3],dim='time')
    else:
        temp = xr.concat([temp_1,temp_2,temp_3],dim='time')
    temp = temp.assign_coords( time = temp.time.dt.year.values, ensemble = i-1 )
    t500 = xr.concat([t500, temp], dim='ensemble')

    # z500
z500_0 = xr.load_dataset(path + "z500max_year_IPSL-CM6A-LR_historical-EXT_r1_1850-1949.nc")['zg'].squeeze() 
z500_1 = xr.load_dataset(path + "z500max_year_IPSL-CM6A-LR_historical-EXT_r1_1950-2014.nc")['zg'].squeeze() 
z500_2 = xr.load_dataset(path + "z500max_year_IPSL-CM6A-LR_historical-EXT_r1_2015-2029.nc")['zg'].squeeze() 
z500_3 = xr.load_dataset(path + "z500max_year_IPSL-CM6A-LR_historical-EXT_r1_2030-2059.nc")['zg'].squeeze() 
z500 = xr.concat([z500_0,z500_1,z500_2,z500_3],dim='time')
z500 = z500.assign_coords( time = z500.time.dt.year.values, ensemble = 1 )
for i in range(3,33):
    temp_1 = xr.load_dataset(path + "z500max_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_1950-2014.nc")['zg'].squeeze() 
    temp_2 = xr.load_dataset(path + "z500max_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2015-2029.nc")['zg'].squeeze() 
    temp_3 = xr.load_dataset(path + "z500max_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2030-2059.nc")['zg'].squeeze() 
    if i in [4,5,6,7,8,9,10,11]:
        temp_0 = xr.load_dataset(path + "z500max_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_1850-1949.nc")['zg'].squeeze() 
        temp = xr.concat([temp_0,temp_1,temp_2,temp_3],dim='time')
    else:
        temp = xr.concat([temp_1,temp_2,temp_3],dim='time')
    temp = temp.assign_coords( time = temp.time.dt.year.values, ensemble = i-1 )
    z500 = xr.concat([z500, temp], dim='ensemble')

    # q2m
q2m_1 = xr.load_dataset(path + "hussmin_year_IPSL-CM6A-LR_historical-EXT_r1_1850-2014.nc")['huss'].squeeze() 
q2m_2 = xr.load_dataset(path + "hussmin_year_IPSL-CM6A-LR_historical-EXT_r1_2015-2029.nc")['huss'].squeeze() 
q2m_3 = xr.load_dataset(path + "hussmin_year_IPSL-CM6A-LR_historical-EXT_r1_2030-2059.nc")['huss'].squeeze() 
q2m = xr.concat([q2m_1,q2m_2,q2m_3],dim='time')
q2m = q2m.assign_coords( time = q2m.time.dt.year.values, ensemble = 1 )
for i in range(3,33):
    temp_1 = xr.load_dataset(path + "hussmin_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_1850-2014.nc")['huss'].squeeze() 
    temp_2 = xr.load_dataset(path + "hussmin_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2015-2029.nc")['huss'].squeeze() 
    temp_3 = xr.load_dataset(path + "hussmin_year_IPSL-CM6A-LR_historical-EXT_r"+str(i)+"_2030-2059.nc")['huss'].squeeze() 
    temp = xr.concat([temp_1,temp_2,temp_3],dim='time')
    temp = temp.assign_coords( time = temp.time.dt.year.values, ensemble = i-1 )
    q2m = xr.concat([q2m, temp], dim='ensemble')

    # orography
orog = xr.load_dataarray( path + "orog_fx_IPSL-CM6A-LR_historical_r1i1p1f1_gr.nc").sel(lat = slice(30,80))
orog.coords['lon'] = (orog.coords['lon'] + 180) % 360 - 180
orog = orog.sortby(orog['lon'])


In [ ]:
## Grid point in WE - GMST

import statsmodels.api as sm
import warnings
import time
warnings.filterwarnings('ignore') 
import signal

## Function to stop if too long

def timeout_handler(num, stack):
    raise Exception("Too long")
    

## Physical upper bound functions

def compute_Tsmax( t500 , z500 , zs , qs ):
    # Everything is in SI units
    Lv = 2.5008*1e6
    cp = 1004.7090
    g = 9.81
    epsilon = 0.0180153/0.028964

    qsat = epsilon * 6.11 * np.exp( Lv / 461.52 * ( 1 / 273.15 - 1 / t500 ) ) / 500

    return t500 + Lv/cp * ( qsat - qs ) + g/cp * ( z500 - zs )

def compute_linear_dependence_physical_upper_bound( covariable , t500 , z500 , q2m , orog ):  
    ## Check if there is trend in q2m
        # Trend in q2m
    cov_reg = sm.add_constant(covariable)
    model_q2m = sm.OLS( q2m , cov_reg )
    results_q2m = model_q2m.fit()
    if results_q2m.pvalues[1] < 0.05 :
        arg = np.argmin( q2m - results_q2m.params[1] * covariable )
        q2m_tsmax = q2m[arg] + results_q2m.params[1] * ( covariable - covariable[arg] )
    else:
        q2m_tsmax = np.min( q2m[q2m != 0] ) # To avoid the issue of non realistic q2m values
    
    ## Compute table of physical upper bounds
    b_phy_reg = compute_Tsmax( t500 , z500 , orog , q2m_tsmax ) - 273.15
    
    ## Prepare covariable for regression
    cov_reg = sm.add_constant(covariable)
    
    ## Linear regression
    model = sm.OLS(b_phy_reg, cov_reg)
    results = model.fit()

    return [ results.params[0] , results.params[1] ]


## Compute fits

def compute_fit_m0( Y , init_fits = None ):
    ## M0: stationary GEV
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y )
    else:
        gev.fit( Y , init = init_fits[0] )
    return np.array(gev.coef_)

def compute_fit_m1( Y , X , init_fits = None ):
    ## M1: non stationary GEV, loc only
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y , c_loc = X )
    else:
        gev.fit( Y , c_loc = X , init = init_fits[1] )
    return np.array(gev.coef_)

def compute_fit_m2( Y , X , init_fits = None ):
    ## M2: non stationary GEV, loc and shape
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVNonStationaryLogExp() )
    else:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVNonStationaryLogExp() , init = init_fits[2] )
    return np.array(gev.coef_)

def compute_fit_m0_B( Y , X , params_linear_b_phy , init_fits = None ):
    ## M0_B: stationary GEV + bound imposed
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * X
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y , l_global = GEVFixedBoundStationaryLink( bound ) )
    else:
        gev.fit( Y , l_global = GEVFixedBoundStationaryLink( bound ) , init = init_fits[3] )
    return np.array(gev.coef_)

def compute_fit_m1_B( Y , X , params_linear_b_phy , init_fits = None ):
    ## M1_B: stationary GEV + bound imposed
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * X
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVFixedBoundLinearLink_alt( bound ) )
    else:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVFixedBoundLinearLink_alt( bound ) , init = init_fits[4] )
    return np.array(gev.coef_)

def compute_fit_m2_B( Y , X , params_linear_b_phy , init_fits = None ):
    ## M2_B: stationary GEV + bound imposed
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * X
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVFixedBoundLinearLink( bound ) )
    else:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVFixedBoundLinearLink( bound ) , init = init_fits[5] )
    return np.array(gev.coef_)

def compute_fits( data , covariable , t500 , z500 , q2m , orog , year_limit , n_years , init_fits , counter_limit = 5 ):
    ## Split data
    data_fit = data.sel( time = slice( year_limit - (n_years-1) , year_limit ) ).values.flatten()
    covariable_fit = np.array( [ covariable.sel( time = slice( year_limit - (n_years-1) , year_limit ) ).values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Fit models
    counter = 0
    while counter < counter_limit:
        counter += 1
        ## Resampling
        idx = np.random.choice( np.arange(data_fit.size) , n_years , replace=False )
        Y = data_fit[idx]
        X = covariable_fit[idx] + 0 * np.ones(n_years)
        
        ## Compute linear dependence of the physical upper bound with the covariable for the data resampled
        params_linear_b_phy = compute_linear_dependence_physical_upper_bound( X , t500[idx] , z500[idx] , q2m[idx] , orog )
        
        ## Compute fits
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(1)
        
        try:
            result_m0 = compute_fit_m0( Y , init_fits = init_fits )
            result_m1 = compute_fit_m1( Y , X , init_fits = init_fits )
            result_m2 = compute_fit_m2( Y , X , init_fits = init_fits )
            result_m0_B = compute_fit_m0_B( Y , X , params_linear_b_phy , init_fits = init_fits )
            result_m1_B = compute_fit_m1_B( Y , X , params_linear_b_phy , init_fits = init_fits )
            result_m2_B = compute_fit_m2_B( Y , X , params_linear_b_phy , init_fits = init_fits )
            
        except:
            signal.alarm(0)
            continue
        else:
            signal.alarm(0)
            break
        
    return [ counter , params_linear_b_phy , result_m0 , result_m1 , result_m2 , result_m0_B , result_m1_B , result_m2_B ]


## Compute metrics

def compute_metrics_m0( parameters , data , covariable ):
    ## Data test
    data_test = data.values.flatten()
    
    ## Parameters of the fitted GEV
    mu, sigma, xi = parameters
    bound = mu - sigma / xi
    if xi < 0:
        bound_tab = np.array([ bound for _ in covariable ])
    else:
        bound_tab = np.array([ np.inf for _ in covariable ])
    
    ## Probability to be above the bound
    if xi < 0:
        result_above_bound = np.mean( data_test > bound )
    else:
        result_above_bound = np.nan
        
    ## Return time max time serie
    max_value = np.max(data_test)
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi , loc = mu , scale = sigma )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi , loc = mu , scale = sigma ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )


    ## Return time for 100 years return level
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi , loc = mu , scale = sigma )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )
            
    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab

def compute_metrics_m1( parameters , data , covariable ):
    ## Data test
    data_test = data.values.flatten()
    covariable_test = np.array( [ covariable.values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Parameters of the fitted GEV
    mu0, mu1, sigma, xi = parameters
    bound = mu0 + mu1 * covariable_test - sigma / xi
    if xi < 0:
        bound_tab = mu0 + mu1 * covariable - sigma / xi
    else:
        bound_tab = np.array([ np.inf for _ in covariable ])
    
    ## Probability to be above the bound
    if xi < 0:
        result_above_bound = np.mean( data_test > bound )
    else:
        result_above_bound = np.nan
        
    ## Return time max time serie
    max_value = np.max(data_test)
    covariable_max_value = covariable_test[ np.argmax(data_test) ]
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi , loc = mu0 + mu1 * covariable_max_value , scale = sigma )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    loc_norms_tab = mu0 + mu1 * covariable.values
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi , loc = loc_norms_tab , scale = sigma ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )


    ## Return time for 100 years return level
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi , loc = mu0 + mu1 * covariable_test , scale = sigma )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )
                
    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab

def compute_metrics_m2( parameters , data , covariable ):
    ## Data test
    data_test = data.values.flatten()
    covariable_test = np.array( [ covariable.values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Parameters of the fitted GEV
    mu0, mu1, sigma0, sigma1, xi = parameters
    bound = mu0 + mu1 * covariable_test - np.log( 1 + np.exp( sigma0 + sigma1 * covariable_test ) ) / xi
    if xi < 0:
        bound_tab = mu0 + mu1 * covariable - np.log( 1 + np.exp( sigma0 + sigma1 * covariable ) ) / xi
    else:
        bound_tab = np.array([ np.inf for _ in covariable ])
    
    ## Probability to be above the bound
    if xi < 0:
        result_above_bound = np.mean( data_test > bound )
    else:
        result_above_bound = np.nan
        
    ## Return time max time serie
    max_value = np.max(data_test)
    covariable_max_value = covariable_test[ np.argmax(data_test) ]
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi , loc = mu0 + mu1 * covariable_max_value , scale = np.log( 1 + np.exp( sigma0 + sigma1 * covariable_max_value ) ) )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    scale_norms_tab = np.log( 1 + np.exp( sigma0 + sigma1 * covariable.values ) )
    loc_norms_tab = mu0 + mu1 * covariable.values
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi , loc = loc_norms_tab , scale = scale_norms_tab ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )


    ## Return time for 100 years return level
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi , loc = mu0 + mu1 * covariable_test , scale = np.log( 1 + np.exp( sigma0 + sigma1 * covariable_test ) ) )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )
            
    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab

def compute_metrics_m0_B( parameters , data , covariable , params_linear_b_phy ):
    ## Data test
    data_test = data.values.flatten()
    covariable_test = np.array( [ covariable.values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Parameters of the fitted GEV
    mu, sigma = parameters
    sigma = np.exp( sigma )
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable_test
    bound_tab = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable
    
    ## Probability to be above the bound
    result_above_bound = np.mean( data_test > bound )
        
    ## Return time max time serie
    max_value = np.max(data_test)
    covariable_max_value = covariable_test[ np.argmax(data_test) ]
    xi_max_value = sigma / ( mu - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_max_value )
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi_max_value , loc = mu , scale = sigma )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    xi_norms_tab = sigma / ( mu - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable.values )
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi_norms_tab , loc = mu , scale = sigma ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )


    ## Return time for 100 years return level
    xi_return_levels = sigma / ( mu - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_test )
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi_return_levels , loc = mu , scale = sigma )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )
     
    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab

def compute_metrics_m1_B( parameters , data , covariable , params_linear_b_phy ):
    ## Data test
    data_test = data.values.flatten()
    covariable_test = np.array( [ covariable.values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Parameters of the fitted GEV
    mu0, mu1, sigma = parameters
    sigma = np.exp( sigma )
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable_test
    bound_tab = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable
    
    ## Probability to be above the bound
    result_above_bound = np.mean( data_test > bound )
        
    ## Return time max time serie
    max_value = np.max(data_test)
    covariable_max_value = covariable_test[ np.argmax(data_test) ]
    xi_max_value = sigma / ( mu0 + mu1 * covariable_max_value  - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_max_value )
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi_max_value , loc = mu0 + mu1 * covariable_max_value , scale = sigma )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    loc_norms_tab = mu0 + mu1 * covariable.values
    xi_norms_tab = sigma / ( loc_norms_tab - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable.values )
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi_norms_tab , loc = loc_norms_tab , scale = sigma ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )


    ## Return time for 100 years return level
    xi_return_levels = sigma / ( mu0 + mu1 * covariable_test - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_test )
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi_return_levels , loc = mu0 + mu1 * covariable_test , scale = sigma )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )
        
    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab


def compute_metrics_m2_B( parameters , data , covariable , params_linear_b_phy ):
    ## Data test
    data_test = data.values.flatten()
    covariable_test = np.array( [ covariable.values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Parameters of the fitted GEV
    mu0, mu1, sigma0, sigma1 = parameters
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable_test
    bound_tab = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable

    ## Probability to be above the bound
    result_above_bound = np.mean( data_test > bound )
    
    ## Return time max time serie
    max_value = np.max(data_test)
    covariable_max_value = covariable_test[ np.argmax(data_test) ]
    scale_max_value = np.log( 1 + np.exp( sigma0 + sigma1 * covariable_max_value ) )
    xi_max_value = scale_max_value / ( mu0 + mu1 * covariable_max_value  - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_max_value )
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi_max_value , loc = mu0 + mu1 * covariable_max_value , scale = scale_max_value )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    scale_norms_tab = np.log( 1 + np.exp( sigma0 + sigma1 * covariable.values ) )
    loc_norms_tab = mu0 + mu1 * covariable.values
    xi_norms_tab = scale_norms_tab / ( loc_norms_tab - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable.values )
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi_norms_tab , loc = loc_norms_tab , scale = scale_norms_tab ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )

    ## Return time for 100 years return level
    scale_return_levels = np.log( 1 + np.exp( sigma0 + sigma1 * covariable_test ) )
    xi_return_levels = scale_return_levels / ( mu0 + mu1 * covariable_test - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_test )
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi_return_levels , loc = mu0 + mu1 * covariable_test , scale = scale_return_levels )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )

    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab


def compute_metrics( fits , data , covariable , params_linear_b_phy ):
    ## Compute metrics
    result_m0 , bound_m0 = compute_metrics_m0( fits[0] , data , covariable )
    result_m1 , bound_m1 = compute_metrics_m1( fits[1] , data , covariable )
    result_m2 , bound_m2 = compute_metrics_m2( fits[2] , data , covariable )
    result_m0_B , bound_m0_B = compute_metrics_m0_B( fits[3] , data , covariable , params_linear_b_phy )
    result_m1_B , bound_m1_B = compute_metrics_m1_B( fits[4] , data , covariable , params_linear_b_phy )
    result_m2_B , bound_m2_B = compute_metrics_m2_B( fits[5] , data , covariable , params_linear_b_phy )

    return [ result_m0 , result_m1 , result_m2 , result_m0_B , result_m1_B , result_m2_B ] , np.array([ bound_m0 , bound_m1 , bound_m2 , bound_m0_B , bound_m1_B , bound_m2_B ])


## Integration function for one grid point: compute fits and then metrics and return results, function to be parallelized

def compute_fits_and_metrics( tas_global , tas_max , t500 , z500 , q2m , orog , sea_filter , id_lat, id_lon , year_limit , n_years , n_test=1000 ):
    ## Sea filter
    #if sea_filter.isel( lat = id_lat , lon = id_lon) == 0:
    #    return np.array([ np.nan for _ in range(n_test) ]) , np.array([ [ np.nan for _ in range(5) ] for _ in range(6) ])
    
    ## Define data for grid point
    covariable = tas_global.mean(dim='ensemble') - tas_global.mean(dim='ensemble').sel(time=slice(1951,1980)).mean()
    tas_max_gp = tas_max.isel( lat = id_lat , lon = id_lon)

    t500_gp = t500.isel( lat = id_lat , lon = id_lon).sel( time = slice( year_limit - (n_years-1) , year_limit ) ).values.copy()
    z500_gp = z500.isel( lat = id_lat , lon = id_lon).sel( time = slice( year_limit - (n_years-1) , year_limit ) ).values.copy()
    for j in range(1950 - (year_limit - (n_years-1))): # Missing values between 1850 and 1949: replace by a random other year
        t500_gp[np.isnan(t500_gp[:,j]),j] = np.random.choice(t500_gp[~np.isnan(t500_gp[:,j]),j], size = np.sum( np.isnan(t500_gp[:,j]) ) , replace = True )
        z500_gp[np.isnan(z500_gp[:,j]),j] = np.random.choice(z500_gp[~np.isnan(z500_gp[:,j]),j], size = np.sum( np.isnan(z500_gp[:,j]) ) , replace = True )
    t500_gp = t500_gp.flatten()
    z500_gp = z500_gp.flatten()
    q2m_gp = q2m.isel( lat = id_lat , lon = id_lon).sel( time = slice( year_limit - (n_years-1) , year_limit ) ).values.flatten()
    q2m_gp[ q2m_gp < 0 ] = np.random.choice( q2m_gp[q2m_gp >= 0] , size = np.sum( q2m_gp < 0 ) )
    orog_gp = orog.isel( lat = id_lat , lon = id_lon).values
    
    ## Results
    counter_tab = []
    metrics_tab = []
    upper_bound_tab = []

    ## Compute first fit and metrics
    fits_ini = compute_fits( tas_max_gp , covariable , t500_gp , z500_gp , q2m_gp , orog_gp , year_limit , n_years , init_fits = None)
    counter_tab.append( fits_ini[0] )
    metrics_ini = compute_metrics( fits_ini[2:] , tas_max_gp , covariable , fits_ini[1] )
    metrics_tab.append( metrics_ini[0] )
    upper_bound_tab.append( metrics_ini[1] )

    ## Compute other fits
    for t in range(1 , n_test):
        
        ## Compute fits
        fits = compute_fits( tas_max_gp , covariable , t500_gp , z500_gp , q2m_gp , orog_gp , year_limit , n_years , init_fits = fits_ini[2:])
        counter_tab.append( fits[0] )

        ## Compute metrics
        metrics = compute_metrics( fits[2:] , tas_max_gp , covariable , fits[1] )
        metrics_tab.append( metrics[0] )
        upper_bound_tab.append( metrics[1] )
    
    return np.array(counter_tab) , np.array(metrics_tab) , np.array(upper_bound_tab)



## Covariable
covariable = tas_global.mean(dim='ensemble')
covariable = covariable - covariable.sel( time = slice("1951","1980") ).mean()

## Parameters
    # Figure paper: id_lat=15, id_lon=72
    # West Russia: id_lat=21, id_lon=91
    # Eastern Kazakstan: id_lat=15, id_lon=104

id_lat = 15
id_lon = 72
year_limit = 2014
n_years = 70

## Compute
metrics = compute_fits_and_metrics( tas_global , tas_max , t500 , z500 , q2m , orog , sea_filter , id_lat, id_lon , year_limit , n_years , n_test=1000 )

## Compute metrics
proba_above_bound = 1 - np.nanmean( metrics[1][:,:,0] == 0 , axis = 0 )
proba_mean_above_bound = np.nanmean( metrics[1][:,:,0] , axis = 0 )
proba_median_above_bound = np.nanmedian( metrics[1][:,:,0] , axis = 0 )

norm_Linf_mean = np.nanmean( metrics[1][:,:,2] , axis = 0 )
norm_Linf_median = np.nanmedian( metrics[1][:,:,2] , axis = 0 )
norm_L2_mean = np.nanmean( metrics[1][:,:,3] , axis = 0 )
norm_L2_median = np.nanmedian( metrics[1][:,:,3] , axis = 0 )

return_time_100_mean = np.nanmean( metrics[1][:,:,4] , axis = 0 )
return_time_100_median = np.nanmedian( metrics[1][:,:,4] , axis = 0 )

phys_bound_05 = np.quantile( metrics[2] , q = 0.05 , axis = 0 )
phys_bound_95 = np.quantile( metrics[2] , q = 0.95 , axis = 0 )
phys_bound_median = np.median( metrics[2] , axis = 0 )


In [ ]:
## Particular fit

member = 14
data_fit = tas_max.isel( lat = id_lat , lon = id_lon ).sel( ensemble = member , time = slice( year_limit-(n_years-1) , year_limit ) )
covariable_fit = covariable.sel( time = slice(year_limit-(n_years-1) , year_limit ) )
t500_gp = t500.isel( lat = id_lat , lon = id_lon).sel( time = slice( year_limit - (n_years-1) , year_limit ) ).copy()
z500_gp = z500.isel( lat = id_lat , lon = id_lon).sel( time = slice( year_limit - (n_years-1) , year_limit ) ).copy()
for j in range(1950 - (year_limit - (n_years-1))): # Missing values between 1850 and 1949: replace by a random other year
    t500_gp[np.isnan(t500_gp[:,j]),j] = np.random.choice(t500_gp[~np.isnan(t500_gp[:,j]),j], size = np.sum( np.isnan(t500_gp[:,j]) ).values , replace = True )
    z500_gp[np.isnan(z500_gp[:,j]),j] = np.random.choice(z500_gp[~np.isnan(z500_gp[:,j]),j], size = np.sum( np.isnan(z500_gp[:,j]) ).values , replace = True )
t500_gp = t500_gp.sel( ensemble = member ).values.flatten()
z500_gp = z500_gp.sel( ensemble = member ).values.flatten()
q2m_gp = q2m.isel( lat = id_lat , lon = id_lon).sel( ensemble = member , time = slice( year_limit - (n_years-1) , year_limit ) ).values.flatten()
q2m_gp[ q2m_gp < 0 ] = np.random.choice( q2m_gp[q2m_gp >= 0] , size = np.sum( q2m_gp < 0 ) )
orog_gp = orog.isel( lat = id_lat , lon = id_lon).values

Y = data_fit.values
X = covariable_fit.values + 0 * np.ones(n_years)
        
## Compute linear dependence of the physical upper bound with the covariable for the data resampled
params_linear_b_phy_member14 = compute_linear_dependence_physical_upper_bound( X , t500_gp , z500_gp , q2m_gp , orog_gp )
        
## Compute fits
mu0_1, mu1_1, sigma0_1, xi0_1 = compute_fit_m1( Y , X )
mu0_2, mu1_2, sigma0_2, sigma1_2, xi0_2 = compute_fit_m2( Y , X )
mu0_1_b, mu1_1_b, sigma0_1_b = compute_fit_m1_B( Y , X , params_linear_b_phy_member14 )
mu0_2_b, mu1_2_b, sigma0_2_b, sigma1_2_b = compute_fit_m2_B( Y , X , params_linear_b_phy_member14 )
            

In [ ]:
color1 = ( 112/256 , 160/256 , 205/256 )
color2 = ( 196/256 , 121/256 , 0/256 )

fig = plt.figure(figsize=(7,7), dpi=1200)
gs = fig.add_gridspec(3,3)

## Scatter points
ax = fig.add_subplot(gs[:2, :2])

ax.plot(np.arange(1850,2060), phys_bound_median[1,:], color=color1, linewidth=1.5, label=r"$B_1$")
ax.fill_between(np.arange(1850,2060),phys_bound_05[1,:],phys_bound_95[1,:], color=color1, alpha=0.25)

ax.plot(np.arange(1850,2060), phys_bound_median[2,:], color=color1, linestyle='dashed', linewidth=1.5, label=r"$B_2$")
ax.fill_between(np.arange(1850,2060),phys_bound_05[2,:],phys_bound_95[2,:], color=color1, alpha=0.25)

ax.plot(np.arange(1850,2060), phys_bound_median[4,:], color=color2, linewidth=1.5, label=r"$B_{\phi}$")
ax.fill_between(np.arange(1850,2060),phys_bound_05[4,:],phys_bound_95[4,:], color=color2, alpha=0.25)

data_tas_max = tas_max.isel( lat = id_lat , lon = id_lon)
for i in range(1850,2060):
    ax.scatter([i for _ in range(data_tas_max['ensemble'].size)], data_tas_max.sel(time=i).values, color='black', s=0.25, alpha=0.7, zorder=2)

ax.plot([2019,2060],[data_tas_max.sel(time=slice(2016,2021)).max(),data_tas_max.sel(time=slice(2016,2021)).max()], color='black', linestyle='dotted', linewidth=1)
ax.plot([2019,2019],[0,data_tas_max.sel(time=slice(2016,2021)).max()], color='black', linestyle='dotted', linewidth=1)
ax.scatter([2019],[data_tas_max.sel(time=slice(2016,2021)).max()], color='black', marker=(3, 0))

ax.set_ylim(22,50)
ax.set_xlim(1850,2059)
ax.annotate('', xy=(1945, 45), xytext=(2014, 45), arrowprops=dict(arrowstyle='<->', color='black', linewidth=0.5))
ax.set_ylabel("TX [°C]")
ax.set_xlabel("Time")
ax.grid(linestyle='dotted', linewidth=0.3)
ax.text(0.97, 0.97, 'a', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.legend(loc='upper left',ncol=3)

## Pdf
ax = fig.add_subplot(gs[:2, 2])

x = np.linspace(22,50,1000)
cov = covariable.sel(time=2019).values

ax.axhline(data_tas_max.sel(time=slice(2016,2021)).max(), color='black', linestyle='dotted', linewidth=1)
ax.hist(data_tas_max.sel(time=slice(2016,2021)).values.flatten(),bins='auto',density=True,histtype='step',color='black', orientation='horizontal')

    # M1
mu =  mu0_1 + mu1_1 * cov
sigma = sigma0_1
xi = xi0_1
ax.plot(sc.genextreme.pdf( x , c = -xi , scale = sigma , loc = mu), x, label=r"$\mathcal{M}_1$", color=color1, linewidth=1.5)
ax.axhline(mu - sigma / xi , color=color1, linewidth=1.5)
ax.boxplot([metrics[2][:,1,169]], positions=[0.2], showcaps=False, boxprops = dict(linewidth=1, color=color1), medianprops = dict(linewidth = 1, color=color1), whiskerprops = dict(linewidth=1, color=color1), showfliers=False, widths=0.025)

    # M2
mu =  mu0_2 + mu1_2 * cov
sigma = np.log( 1 + np.exp(sigma0_2 + sigma1_2 * cov)) 
xi = xi0_2
ax.plot(sc.genextreme.pdf( x , c = -xi , scale = sigma , loc = mu), x, label=r"$\mathcal{M}_2$", color=color1, linestyle='dashed', linewidth=1.5)
ax.axhline(mu - sigma / xi , color=color1, linestyle='dashed', linewidth=1.5)
ax.boxplot([metrics[2][:,2,169]], positions=[0.25], showcaps=False, boxprops = dict(linewidth=1, color=color1, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color1, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color1, linestyle='dashed'), showfliers=False, widths=0.025)

    # M1_b
bound = params_linear_b_phy_member14[0] + params_linear_b_phy_member14[1]*cov
mu =  mu0_1_b + mu1_1_b * cov
sigma = np.exp(sigma0_1_b)
xi = - sigma / (bound - mu)
plt.plot(sc.genextreme.pdf( x , c = -xi , scale = sigma , loc = mu), x, label=r"$\mathcal{M}_1^B$", color=color2, linewidth=1.5)
ax.axhline(bound , color=color2, linewidth=1.5)
ax.boxplot([metrics[2][:,4,169]], positions=[0.15], showcaps=False, boxprops = dict(linewidth=1, color=color2), medianprops = dict(linewidth = 1, color=color2), whiskerprops = dict(linewidth=1, color=color2), showfliers=False, widths=0.025)

    # M2_b
bound = params_linear_b_phy_member14[0] + params_linear_b_phy_member14[1]*cov
mu =  mu0_2_b + mu1_2_b * cov
sigma = np.log( 1 + np.exp(sigma0_2_b + sigma1_2_b*cov))
xi = - sigma / (bound - mu)
plt.plot(sc.genextreme.pdf( x , c = -xi , scale = sigma , loc = mu), x, label=r"$\mathcal{M}_2^B$", color=color2, linestyle='dashed', linewidth=1.5)

ax.set_ylim(22,50)
ax.set_ylabel("TX [°C]")
ax.set_xlim(0,0.3)
ax.set_xticks([0,0.1,0.2,0.3])
ax.set_xticklabels([0,0.1,0.2,0.3])
ax.set_xlabel("Density")
ax.grid(linestyle='dotted', linewidth=0.3)
ax.text(0.97, 0.97, 'b', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.legend(loc='upper left',ncol=2, fontsize=7)

## Return time  
ax = fig.add_subplot(gs[2, 0])

ax.boxplot([metrics[1][:,1,4]], positions=[1], showcaps=False, boxprops = dict(linewidth=1, color=color1), medianprops = dict(linewidth = 1, color=color1), whiskerprops = dict(linewidth=1, color=color1), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,2,4]], positions=[2], showcaps=False, boxprops = dict(linewidth=1, color=color1, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color1, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color1, linestyle='dashed'), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,4,4]], positions=[3], showcaps=False, boxprops = dict(linewidth=1, color=color2), medianprops = dict(linewidth = 1, color=color2), whiskerprops = dict(linewidth=1, color=color2), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,5,4]], positions=[4], showcaps=False, boxprops = dict(linewidth=1, color=color2, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color2, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color2, linestyle='dashed'), showfliers=False, widths=0.3)

ax.axhline(100, color='black', linestyle='dashed', linewidth=1)
ax.set_ylabel(r"$\hat{r}(Z > \hat{z}_{p=1\%})$ [y]")
ax.set_xticklabels([r"$\mathcal{M}_1$",r"$\mathcal{M}_2$",r"$\mathcal{M}_1^B$",r"$\mathcal{M}_2^B$"])
ax.grid(linestyle='dotted', linewidth=0.3)
ax.text(0.97, 0.97, 'c', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.set_ylim(0,1000)

## cdf_norm_Linf 
ax = fig.add_subplot(gs[2, 1])

ax.boxplot([metrics[1][:,1,2]], positions=[1], showcaps=False, boxprops = dict(linewidth=1, color=color1), medianprops = dict(linewidth = 1, color=color1), whiskerprops = dict(linewidth=1, color=color1), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,2,2]], positions=[2], showcaps=False, boxprops = dict(linewidth=1, color=color1, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color1, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color1, linestyle='dashed'), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,4,2]], positions=[3], showcaps=False, boxprops = dict(linewidth=1, color=color2), medianprops = dict(linewidth = 1, color=color2), whiskerprops = dict(linewidth=1, color=color2), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,5,2]], positions=[4], showcaps=False, boxprops = dict(linewidth=1, color=color2, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color2, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color2, linestyle='dashed'), showfliers=False, widths=0.3)

ax.set_ylabel(r"$\|\hat{F} - F_e\|_\infty$")
ax.set_xticklabels([r"$\mathcal{M}_1$",r"$\mathcal{M}_2$",r"$\mathcal{M}_1^B$",r"$\mathcal{M}_2^B$"])
ax.grid(linestyle='dotted', linewidth=0.3)
ax.text(0.97, 0.97, 'd', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.set_ylim(bottom=0)

## cdf_norm_L2
ax = fig.add_subplot(gs[2, 2])

ax.boxplot([metrics[1][:,1,3]], positions=[1], showcaps=False, boxprops = dict(linewidth=1, color=color1), medianprops = dict(linewidth = 1, color=color1), whiskerprops = dict(linewidth=1, color=color1), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,2,3]], positions=[2], showcaps=False, boxprops = dict(linewidth=1, color=color1, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color1, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color1, linestyle='dashed'), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,4,3]], positions=[3], showcaps=False, boxprops = dict(linewidth=1, color=color2), medianprops = dict(linewidth = 1, color=color2), whiskerprops = dict(linewidth=1, color=color2), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,5,3]], positions=[4], showcaps=False, boxprops = dict(linewidth=1, color=color2, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color2, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color2, linestyle='dashed'), showfliers=False, widths=0.3)

ax.set_ylabel(r"$\|\hat{F} - F_e\|_2$")
ax.set_xticklabels([r"$\mathcal{M}_1$",r"$\mathcal{M}_2$",r"$\mathcal{M}_1^B$",r"$\mathcal{M}_2^B$"])
ax.grid(linestyle='dotted', linewidth=0.3)
ax.text(0.97, 0.97, 'e', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.show()

In [ ]:
## Grid point in WE - RMST

import statsmodels.api as sm
import warnings
import time
warnings.filterwarnings('ignore') 
import signal

## Function to stop if too long

def timeout_handler(num, stack):
    raise Exception("Too long")
    

## Physical upper bound functions

def compute_Tsmax( t500 , z500 , zs , qs ):
    # Everything is in SI units
    Lv = 2.5008*1e6
    cp = 1004.7090
    g = 9.81
    epsilon = 0.0180153/0.028964

    qsat = epsilon * 6.11 * np.exp( Lv / 461.52 * ( 1 / 273.15 - 1 / t500 ) ) / 500

    return t500 + Lv/cp * ( qsat - qs ) + g/cp * ( z500 - zs )

def compute_linear_dependence_physical_upper_bound( covariable , t500 , z500 , q2m , orog ):  
    ## Check if there is trend in q2m
        # Trend in q2m
    cov_reg = sm.add_constant(covariable)
    model_q2m = sm.OLS( q2m , cov_reg )
    results_q2m = model_q2m.fit()
    if results_q2m.pvalues[1] < 0.05 :
        arg = np.argmin( q2m - results_q2m.params[1] * covariable )
        q2m_tsmax = q2m[arg] + results_q2m.params[1] * ( covariable - covariable[arg] )
    else:
        q2m_tsmax = np.min( q2m[q2m != 0] ) # To avoid the issue of non realistic q2m values
    
    ## Compute table of physical upper bounds
    b_phy_reg = compute_Tsmax( t500 , z500 , orog , q2m_tsmax ) - 273.15
    
    ## Prepare covariable for regression
    cov_reg = sm.add_constant(covariable)
    
    ## Linear regression
    model = sm.OLS(b_phy_reg, cov_reg)
    results = model.fit()

    return [ results.params[0] , results.params[1] ]


## Compute fits

def compute_fit_m0( Y , init_fits = None ):
    ## M0: stationary GEV
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y )
    else:
        gev.fit( Y , init = init_fits[0] )
    return np.array(gev.coef_)

def compute_fit_m1( Y , X , init_fits = None ):
    ## M1: non stationary GEV, loc only
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y , c_loc = X )
    else:
        gev.fit( Y , c_loc = X , init = init_fits[1] )
    return np.array(gev.coef_)

def compute_fit_m2( Y , X , init_fits = None ):
    ## M2: non stationary GEV, loc and shape
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVNonStationaryLogExp() )
    else:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVNonStationaryLogExp() , init = init_fits[2] )
    return np.array(gev.coef_)

def compute_fit_m0_B( Y , X , params_linear_b_phy , init_fits = None ):
    ## M0_B: stationary GEV + bound imposed
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * X
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y , l_global = GEVFixedBoundStationaryLink( bound ) )
    else:
        gev.fit( Y , l_global = GEVFixedBoundStationaryLink( bound ) , init = init_fits[3] )
    return np.array(gev.coef_)

def compute_fit_m1_B( Y , X , params_linear_b_phy , init_fits = None ):
    ## M1_B: stationary GEV + bound imposed
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * X
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVFixedBoundLinearLink_alt( bound ) )
    else:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVFixedBoundLinearLink_alt( bound ) , init = init_fits[4] )
    return np.array(gev.coef_)

def compute_fit_m2_B( Y , X , params_linear_b_phy , init_fits = None ):
    ## M2_B: stationary GEV + bound imposed
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * X
    gev = sd.GEV() 
    if init_fits is None:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVFixedBoundLinearLink( bound ) )
    else:
        gev.fit( Y , c_global = [X.reshape(-1,1)] , l_global = GEVFixedBoundLinearLink( bound ) , init = init_fits[5] )
    return np.array(gev.coef_)

def compute_fits( data , covariable , t500 , z500 , q2m , orog , year_limit , n_years , init_fits , counter_limit = 5 ):
    ## Split data
    data_fit = data.sel( time = slice( year_limit - (n_years-1) , year_limit ) ).values.flatten()
    covariable_fit = np.array( [ covariable.sel( time = slice( year_limit - (n_years-1) , year_limit ) ).values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Fit models
    counter = 0
    while counter < counter_limit:
        counter += 1
        ## Resampling
        idx = np.random.choice( np.arange(data_fit.size) , n_years , replace=False )
        Y = data_fit[idx]
        X = covariable_fit[idx] + 0 * np.ones(n_years)
        
        ## Compute linear dependence of the physical upper bound with the covariable for the data resampled
        params_linear_b_phy = compute_linear_dependence_physical_upper_bound( X , t500[idx] , z500[idx] , q2m[idx] , orog )
        
        ## Compute fits
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(1)
        
        try:
            result_m0 = compute_fit_m0( Y , init_fits = init_fits )
            result_m1 = compute_fit_m1( Y , X , init_fits = init_fits )
            result_m2 = compute_fit_m2( Y , X , init_fits = init_fits )
            result_m0_B = compute_fit_m0_B( Y , X , params_linear_b_phy , init_fits = init_fits )
            result_m1_B = compute_fit_m1_B( Y , X , params_linear_b_phy , init_fits = init_fits )
            result_m2_B = compute_fit_m2_B( Y , X , params_linear_b_phy , init_fits = init_fits )
            
        except:
            signal.alarm(0)
            continue
        else:
            signal.alarm(0)
            break
        
    return [ counter , params_linear_b_phy , result_m0 , result_m1 , result_m2 , result_m0_B , result_m1_B , result_m2_B ]


## Compute metrics

def compute_metrics_m0( parameters , data , covariable ):
    ## Data test
    data_test = data.values.flatten()
    
    ## Parameters of the fitted GEV
    mu, sigma, xi = parameters
    bound = mu - sigma / xi
    if xi < 0:
        bound_tab = np.array([ bound for _ in covariable ])
    else:
        bound_tab = np.array([ np.inf for _ in covariable ])
    
    ## Probability to be above the bound
    if xi < 0:
        result_above_bound = np.mean( data_test > bound )
    else:
        result_above_bound = np.nan
        
    ## Return time max time serie
    max_value = np.max(data_test)
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi , loc = mu , scale = sigma )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi , loc = mu , scale = sigma ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )


    ## Return time for 100 years return level
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi , loc = mu , scale = sigma )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )
            
    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab

def compute_metrics_m1( parameters , data , covariable ):
    ## Data test
    data_test = data.values.flatten()
    covariable_test = np.array( [ covariable.values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Parameters of the fitted GEV
    mu0, mu1, sigma, xi = parameters
    bound = mu0 + mu1 * covariable_test - sigma / xi
    if xi < 0:
        bound_tab = mu0 + mu1 * covariable - sigma / xi
    else:
        bound_tab = np.array([ np.inf for _ in covariable ])
    
    ## Probability to be above the bound
    if xi < 0:
        result_above_bound = np.mean( data_test > bound )
    else:
        result_above_bound = np.nan
        
    ## Return time max time serie
    max_value = np.max(data_test)
    covariable_max_value = covariable_test[ np.argmax(data_test) ]
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi , loc = mu0 + mu1 * covariable_max_value , scale = sigma )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    loc_norms_tab = mu0 + mu1 * covariable.values
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi , loc = loc_norms_tab , scale = sigma ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )


    ## Return time for 100 years return level
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi , loc = mu0 + mu1 * covariable_test , scale = sigma )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )
                
    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab

def compute_metrics_m2( parameters , data , covariable ):
    ## Data test
    data_test = data.values.flatten()
    covariable_test = np.array( [ covariable.values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Parameters of the fitted GEV
    mu0, mu1, sigma0, sigma1, xi = parameters
    bound = mu0 + mu1 * covariable_test - np.log( 1 + np.exp( sigma0 + sigma1 * covariable_test ) ) / xi
    if xi < 0:
        bound_tab = mu0 + mu1 * covariable - np.log( 1 + np.exp( sigma0 + sigma1 * covariable ) ) / xi
    else:
        bound_tab = np.array([ np.inf for _ in covariable ])
    
    ## Probability to be above the bound
    if xi < 0:
        result_above_bound = np.mean( data_test > bound )
    else:
        result_above_bound = np.nan
        
    ## Return time max time serie
    max_value = np.max(data_test)
    covariable_max_value = covariable_test[ np.argmax(data_test) ]
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi , loc = mu0 + mu1 * covariable_max_value , scale = np.log( 1 + np.exp( sigma0 + sigma1 * covariable_max_value ) ) )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    scale_norms_tab = np.log( 1 + np.exp( sigma0 + sigma1 * covariable.values ) )
    loc_norms_tab = mu0 + mu1 * covariable.values
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi , loc = loc_norms_tab , scale = scale_norms_tab ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )


    ## Return time for 100 years return level
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi , loc = mu0 + mu1 * covariable_test , scale = np.log( 1 + np.exp( sigma0 + sigma1 * covariable_test ) ) )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )
            
    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab

def compute_metrics_m0_B( parameters , data , covariable , params_linear_b_phy ):
    ## Data test
    data_test = data.values.flatten()
    covariable_test = np.array( [ covariable.values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Parameters of the fitted GEV
    mu, sigma = parameters
    sigma = np.exp( sigma )
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable_test
    bound_tab = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable
    
    ## Probability to be above the bound
    result_above_bound = np.mean( data_test > bound )
        
    ## Return time max time serie
    max_value = np.max(data_test)
    covariable_max_value = covariable_test[ np.argmax(data_test) ]
    xi_max_value = sigma / ( mu - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_max_value )
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi_max_value , loc = mu , scale = sigma )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    xi_norms_tab = sigma / ( mu - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable.values )
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi_norms_tab , loc = mu , scale = sigma ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )


    ## Return time for 100 years return level
    xi_return_levels = sigma / ( mu - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_test )
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi_return_levels , loc = mu , scale = sigma )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )
     
    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab

def compute_metrics_m1_B( parameters , data , covariable , params_linear_b_phy ):
    ## Data test
    data_test = data.values.flatten()
    covariable_test = np.array( [ covariable.values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Parameters of the fitted GEV
    mu0, mu1, sigma = parameters
    sigma = np.exp( sigma )
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable_test
    bound_tab = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable
    
    ## Probability to be above the bound
    result_above_bound = np.mean( data_test > bound )
        
    ## Return time max time serie
    max_value = np.max(data_test)
    covariable_max_value = covariable_test[ np.argmax(data_test) ]
    xi_max_value = sigma / ( mu0 + mu1 * covariable_max_value  - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_max_value )
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi_max_value , loc = mu0 + mu1 * covariable_max_value , scale = sigma )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    loc_norms_tab = mu0 + mu1 * covariable.values
    xi_norms_tab = sigma / ( loc_norms_tab - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable.values )
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi_norms_tab , loc = loc_norms_tab , scale = sigma ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )


    ## Return time for 100 years return level
    xi_return_levels = sigma / ( mu0 + mu1 * covariable_test - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_test )
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi_return_levels , loc = mu0 + mu1 * covariable_test , scale = sigma )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )
        
    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab


def compute_metrics_m2_B( parameters , data , covariable , params_linear_b_phy ):
    ## Data test
    data_test = data.values.flatten()
    covariable_test = np.array( [ covariable.values for _ in range(data['ensemble'].size) ] ).flatten()
    
    ## Parameters of the fitted GEV
    mu0, mu1, sigma0, sigma1 = parameters
    bound = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable_test
    bound_tab = params_linear_b_phy[0] + params_linear_b_phy[1] * covariable

    ## Probability to be above the bound
    result_above_bound = np.mean( data_test > bound )
    
    ## Return time max time serie
    max_value = np.max(data_test)
    covariable_max_value = covariable_test[ np.argmax(data_test) ]
    scale_max_value = np.log( 1 + np.exp( sigma0 + sigma1 * covariable_max_value ) )
    xi_max_value = scale_max_value / ( mu0 + mu1 * covariable_max_value  - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_max_value )
    with np.errstate(divide='ignore'):
        result_return_time_max = 1 / ( 1 - sc.genextreme.cdf( max_value , c = -xi_max_value , loc = mu0 + mu1 * covariable_max_value , scale = scale_max_value )) 
    
    ## L2 and Linf each year averaged over time
    y_ref = np.arange( 1 , data['ensemble'].size + 1 ) / data['ensemble'].size 
    y_ref = np.repeat( [y_ref] , data['time'].size , axis = 0 ).T # (31,210)
    
    scale_norms_tab = np.log( 1 + np.exp( sigma0 + sigma1 * covariable.values ) )
    loc_norms_tab = mu0 + mu1 * covariable.values
    xi_norms_tab = scale_norms_tab / ( loc_norms_tab - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable.values )
    data_norms_tab = np.sort( data.values , axis = 0 ) # (31,210)
    
    y_fit = sc.genextreme.cdf( data_norms_tab , c = -xi_norms_tab , loc = loc_norms_tab , scale = scale_norms_tab ) 
    result_norm_Linf = np.mean( np.max( np.abs( y_ref - y_fit ) , axis = 0 ) )
    result_norm_L2 = np.mean( np.sqrt( np.sum( ( y_ref - y_fit )**2 / data['ensemble'].size , axis = 0 ) ) )

    ## Return time for 100 years return level
    scale_return_levels = np.log( 1 + np.exp( sigma0 + sigma1 * covariable_test ) )
    xi_return_levels = scale_return_levels / ( mu0 + mu1 * covariable_test - params_linear_b_phy[0] - params_linear_b_phy[1] * covariable_test )
    return_levels = sc.genextreme.ppf( 1 - 1/100 , c = -xi_return_levels , loc = mu0 + mu1 * covariable_test , scale = scale_return_levels )
    with np.errstate(divide='ignore'):
        result_return_time = 1 / np.mean( data_test > return_levels )

    return [ result_above_bound , result_return_time_max , result_norm_Linf , result_norm_L2 , result_return_time ] , bound_tab


def compute_metrics( fits , data , covariable , params_linear_b_phy ):
    ## Compute metrics
    result_m0 , bound_m0 = compute_metrics_m0( fits[0] , data , covariable )
    result_m1 , bound_m1 = compute_metrics_m1( fits[1] , data , covariable )
    result_m2 , bound_m2 = compute_metrics_m2( fits[2] , data , covariable )
    result_m0_B , bound_m0_B = compute_metrics_m0_B( fits[3] , data , covariable , params_linear_b_phy )
    result_m1_B , bound_m1_B = compute_metrics_m1_B( fits[4] , data , covariable , params_linear_b_phy )
    result_m2_B , bound_m2_B = compute_metrics_m2_B( fits[5] , data , covariable , params_linear_b_phy )

    return [ result_m0 , result_m1 , result_m2 , result_m0_B , result_m1_B , result_m2_B ] , np.array([ bound_m0 , bound_m1 , bound_m2 , bound_m0_B , bound_m1_B , bound_m2_B ])


## Integration function for one grid point: compute fits and then metrics and return results, function to be parallelized

def compute_fits_and_metrics( tas_global , tas_max , t500 , z500 , q2m , orog , sea_filter , id_lat, id_lon , year_limit , n_years , n_test=1000 ):
    ## Sea filter
    #if sea_filter.isel( lat = id_lat , lon = id_lon) == 0:
    #    return np.array([ np.nan for _ in range(n_test) ]) , np.array([ [ np.nan for _ in range(5) ] for _ in range(6) ])
    
    ## Define data for grid point
    covariable = tas_global.mean(dim='ensemble') - tas_global.mean(dim='ensemble').sel(time=slice(1951,1980)).mean()
    tas_max_gp = tas_max.isel( lat = id_lat , lon = id_lon)

    t500_gp = t500.isel( lat = id_lat , lon = id_lon).sel( time = slice( year_limit - (n_years-1) , year_limit ) ).values.copy()
    z500_gp = z500.isel( lat = id_lat , lon = id_lon).sel( time = slice( year_limit - (n_years-1) , year_limit ) ).values.copy()
    for j in range(1950 - (year_limit - (n_years-1))): # Missing values between 1850 and 1949: replace by a random other year
        t500_gp[np.isnan(t500_gp[:,j]),j] = np.random.choice(t500_gp[~np.isnan(t500_gp[:,j]),j], size = np.sum( np.isnan(t500_gp[:,j]) ) , replace = True )
        z500_gp[np.isnan(z500_gp[:,j]),j] = np.random.choice(z500_gp[~np.isnan(z500_gp[:,j]),j], size = np.sum( np.isnan(z500_gp[:,j]) ) , replace = True )
    t500_gp = t500_gp.flatten()
    z500_gp = z500_gp.flatten()
    q2m_gp = q2m.isel( lat = id_lat , lon = id_lon).sel( time = slice( year_limit - (n_years-1) , year_limit ) ).values.flatten()
    q2m_gp[ q2m_gp < 0 ] = np.random.choice( q2m_gp[q2m_gp >= 0] , size = np.sum( q2m_gp < 0 ) )
    orog_gp = orog.isel( lat = id_lat , lon = id_lon).values
    
    ## Results
    counter_tab = []
    metrics_tab = []
    upper_bound_tab = []

    ## Compute first fit and metrics
    fits_ini = compute_fits( tas_max_gp , covariable , t500_gp , z500_gp , q2m_gp , orog_gp , year_limit , n_years , init_fits = None)
    counter_tab.append( fits_ini[0] )
    metrics_ini = compute_metrics( fits_ini[2:] , tas_max_gp , covariable , fits_ini[1] )
    metrics_tab.append( metrics_ini[0] )
    upper_bound_tab.append( metrics_ini[1] )

    ## Compute other fits
    for t in range(1 , n_test):
        
        ## Compute fits
        fits = compute_fits( tas_max_gp , covariable , t500_gp , z500_gp , q2m_gp , orog_gp , year_limit , n_years , init_fits = fits_ini[2:])
        counter_tab.append( fits[0] )

        ## Compute metrics
        metrics = compute_metrics( fits[2:] , tas_max_gp , covariable , fits[1] )
        metrics_tab.append( metrics[0] )
        upper_bound_tab.append( metrics[1] )
    
    return np.array(counter_tab) , np.array(metrics_tab) , np.array(upper_bound_tab)



## Parameters
id_lat = 15
id_lon = 72
year_limit = 2014
n_years = 70

## Covariable
longitude = tas_mean['lon']
if longitude[id_lon] < -30:
    cov = tas_mean.sel(lon=slice(-180,-30))
elif ((longitude[id_lon] >= -30) and (longitude[id_lon] < 50)):
    cov = tas_mean.sel(lon=slice(-30,50))
elif longitude[id_lon] >=50:
    cov = tas_mean.sel(lon=slice(50,180))
weights = np.cos(np.deg2rad(cov.lat))
cov = cov.weighted(weights).mean(("lon","lat"))
covariable = cov.mean(dim='ensemble')
covariable = covariable - covariable.sel( time = slice("1951","1980") ).mean()

## Compute
metrics = compute_fits_and_metrics( cov , tas_max , t500 , z500 , q2m , orog , sea_filter , id_lat, id_lon , year_limit , n_years , n_test=1000 )

## Compute metrics
proba_above_bound = 1 - np.nanmean( metrics[1][:,:,0] == 0 , axis = 0 )
proba_mean_above_bound = np.nanmean( metrics[1][:,:,0] , axis = 0 )
proba_median_above_bound = np.nanmedian( metrics[1][:,:,0] , axis = 0 )

norm_Linf_mean = np.nanmean( metrics[1][:,:,2] , axis = 0 )
norm_Linf_median = np.nanmedian( metrics[1][:,:,2] , axis = 0 )
norm_L2_mean = np.nanmean( metrics[1][:,:,3] , axis = 0 )
norm_L2_median = np.nanmedian( metrics[1][:,:,3] , axis = 0 )

return_time_100_mean = np.nanmean( metrics[1][:,:,4] , axis = 0 )
return_time_100_median = np.nanmedian( metrics[1][:,:,4] , axis = 0 )

phys_bound_05 = np.quantile( metrics[2] , q = 0.05 , axis = 0 )
phys_bound_95 = np.quantile( metrics[2] , q = 0.95 , axis = 0 )
phys_bound_median = np.median( metrics[2] , axis = 0 )

In [ ]:
## Particular fit

member = 14
data_fit = tas_max.isel( lat = id_lat , lon = id_lon ).sel( ensemble = member , time = slice( year_limit-(n_years-1) , year_limit ) )
covariable_fit = covariable.sel( time = slice(year_limit-(n_years-1) , year_limit ) )
t500_gp = t500.isel( lat = id_lat , lon = id_lon).sel( time = slice( year_limit - (n_years-1) , year_limit ) ).copy()
z500_gp = z500.isel( lat = id_lat , lon = id_lon).sel( time = slice( year_limit - (n_years-1) , year_limit ) ).copy()
for j in range(1950 - (year_limit - (n_years-1))): # Missing values between 1850 and 1949: replace by a random other year
    t500_gp[np.isnan(t500_gp[:,j]),j] = np.random.choice(t500_gp[~np.isnan(t500_gp[:,j]),j], size = np.sum( np.isnan(t500_gp[:,j]) ).values , replace = True )
    z500_gp[np.isnan(z500_gp[:,j]),j] = np.random.choice(z500_gp[~np.isnan(z500_gp[:,j]),j], size = np.sum( np.isnan(z500_gp[:,j]) ).values , replace = True )
t500_gp = t500_gp.sel( ensemble = member ).values.flatten()
z500_gp = z500_gp.sel( ensemble = member ).values.flatten()
q2m_gp = q2m.isel( lat = id_lat , lon = id_lon).sel( ensemble = member , time = slice( year_limit - (n_years-1) , year_limit ) ).values.flatten()
q2m_gp[ q2m_gp < 0 ] = np.random.choice( q2m_gp[q2m_gp >= 0] , size = np.sum( q2m_gp < 0 ) )
orog_gp = orog.isel( lat = id_lat , lon = id_lon).values

Y = data_fit.values
X = covariable_fit.values + 0 * np.ones(n_years)
        
## Compute linear dependence of the physical upper bound with the covariable for the data resampled
params_linear_b_phy_member14 = compute_linear_dependence_physical_upper_bound( X , t500_gp , z500_gp , q2m_gp , orog_gp )
        
## Compute fits
mu0_1, mu1_1, sigma0_1, xi0_1 = compute_fit_m1( Y , X )
mu0_2, mu1_2, sigma0_2, sigma1_2, xi0_2 = compute_fit_m2( Y , X )
mu0_1_b, mu1_1_b, sigma0_1_b = compute_fit_m1_B( Y , X , params_linear_b_phy_member14 )
mu0_2_b, mu1_2_b, sigma0_2_b, sigma1_2_b = compute_fit_m2_B( Y , X , params_linear_b_phy_member14 )
            

In [ ]:
color1 = ( 112/256 , 160/256 , 205/256 )
color2 = ( 196/256 , 121/256 , 0/256 )

fig = plt.figure(figsize=(7,7), dpi=1200)
gs = fig.add_gridspec(3,3)

## Scatter points
ax = fig.add_subplot(gs[:2, :2])

ax.plot(np.arange(1850,2060), phys_bound_median[1,:], color=color1, linewidth=1.5, label=r"$B_1$")
ax.fill_between(np.arange(1850,2060),phys_bound_05[1,:],phys_bound_95[1,:], color=color1, alpha=0.25)
#ax.plot(np.arange(1850,2060), phys_bound_05[1,:], color=color1, linewidth=0.75, alpha=0.5)
#ax.plot(np.arange(1850,2060), phys_bound_95[1,:], color=color1, linewidth=0.75, alpha=0.5)

ax.plot(np.arange(1850,2060), phys_bound_median[2,:], color=color1, linestyle='dashed', linewidth=1.5, label=r"$B_2$")
ax.fill_between(np.arange(1850,2060),phys_bound_05[2,:],phys_bound_95[2,:], color=color1, alpha=0.25)
#ax.plot(np.arange(1850,2060), phys_bound_05[2,:], color=color1, linestyle='dashed', linewidth=0.5, alpha=0.5)
#ax.plot(np.arange(1850,2060), phys_bound_95[2,:], color=color1, linestyle='dashed', linewidth=0.5, alpha=0.5)

ax.plot(np.arange(1850,2060), phys_bound_median[4,:], color=color2, linewidth=1.5, label=r"$B_{\phi}$")
ax.fill_between(np.arange(1850,2060),phys_bound_05[4,:],phys_bound_95[4,:], color=color2, alpha=0.25)
#ax.plot(np.arange(1850,2060), phys_bound_05[4,:], color=color2, linewidth=0.5, alpha=0.5)
#ax.plot(np.arange(1850,2060), phys_bound_95[4,:], color=color2, linewidth=0.5, alpha=0.5)

data_tas_max = tas_max.isel( lat = id_lat , lon = id_lon)
for i in range(1850,2060):
    ax.scatter([i for _ in range(data_tas_max['ensemble'].size)], data_tas_max.sel(time=i).values, color='black', s=0.25, alpha=0.7, zorder=2)

ax.plot([2019,2060],[data_tas_max.sel(time=slice(2016,2021)).max(),data_tas_max.sel(time=slice(2016,2021)).max()], color='black', linestyle='dotted', linewidth=1)
ax.plot([2019,2019],[0,data_tas_max.sel(time=slice(2016,2021)).max()], color='black', linestyle='dotted', linewidth=1)
ax.scatter([2019],[data_tas_max.sel(time=slice(2016,2021)).max()], color='black', marker=(3, 0))

ax.set_ylim(22,50)
ax.set_xlim(1850,2059)
ax.annotate('', xy=(1945, 45), xytext=(2014, 45), arrowprops=dict(arrowstyle='<->', color='black', linewidth=0.5))
ax.set_ylabel("TX [°C]")
ax.set_xlabel("Time")
ax.grid(linestyle='dotted', linewidth=0.3)
ax.text(0.97, 0.97, 'a', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.legend(loc='upper left',ncol=3)

## Pdf
ax = fig.add_subplot(gs[:2, 2])

x = np.linspace(22,50,1000)
cov = covariable.sel(time=2019).values

ax.axhline(data_tas_max.sel(time=slice(2016,2021)).max(), color='black', linestyle='dotted', linewidth=1)
ax.hist(data_tas_max.sel(time=slice(2016,2021)).values.flatten(),bins='auto',density=True,histtype='step',color='black', orientation='horizontal')

    # M1
mu =  mu0_1 + mu1_1 * cov
sigma = sigma0_1
xi = xi0_1
ax.plot(sc.genextreme.pdf( x , c = -xi , scale = sigma , loc = mu), x, label=r"$\mathcal{M}_1$", color=color1, linewidth=1.5)
ax.axhline(mu - sigma / xi , color=color1, linewidth=1.5)
ax.boxplot([metrics[2][:,1,169]], positions=[0.2], showcaps=False, boxprops = dict(linewidth=1, color=color1), medianprops = dict(linewidth = 1, color=color1), whiskerprops = dict(linewidth=1, color=color1), showfliers=False, widths=0.025)

    # M2
mu =  mu0_2 + mu1_2 * cov
sigma = np.log( 1 + np.exp(sigma0_2 + sigma1_2 * cov)) 
xi = xi0_2
ax.plot(sc.genextreme.pdf( x , c = -xi , scale = sigma , loc = mu), x, label=r"$\mathcal{M}_2$", color=color1, linestyle='dashed', linewidth=1.5)
ax.axhline(mu - sigma / xi , color=color1, linestyle='dashed', linewidth=1.5)
ax.boxplot([metrics[2][:,2,169]], positions=[0.25], showcaps=False, boxprops = dict(linewidth=1, color=color1, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color1, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color1, linestyle='dashed'), showfliers=False, widths=0.025)

    # M1_b
bound = params_linear_b_phy_member14[0] + params_linear_b_phy_member14[1]*cov
mu =  mu0_1_b + mu1_1_b * cov
sigma = np.exp(sigma0_1_b)
xi = - sigma / (bound - mu)
plt.plot(sc.genextreme.pdf( x , c = -xi , scale = sigma , loc = mu), x, label=r"$\mathcal{M}_1^B$", color=color2, linewidth=1.5)
ax.axhline(bound , color=color2, linewidth=1.5)
ax.boxplot([metrics[2][:,4,169]], positions=[0.15], showcaps=False, boxprops = dict(linewidth=1, color=color2), medianprops = dict(linewidth = 1, color=color2), whiskerprops = dict(linewidth=1, color=color2), showfliers=False, widths=0.025)

    # M2_b
bound = params_linear_b_phy_member14[0] + params_linear_b_phy_member14[1]*cov
mu =  mu0_2_b + mu1_2_b * cov
sigma = np.log( 1 + np.exp(sigma0_2_b + sigma1_2_b*cov))
xi = - sigma / (bound - mu)
plt.plot(sc.genextreme.pdf( x , c = -xi , scale = sigma , loc = mu), x, label=r"$\mathcal{M}_2^B$", color=color2, linestyle='dashed', linewidth=1.5)

ax.set_ylim(22,50)
ax.set_ylabel("TX [°C]")
ax.set_xlim(0,0.3)
ax.set_xticks([0,0.1,0.2,0.3])
ax.set_xticklabels([0,0.1,0.2,0.3])
ax.set_xlabel("Density")
ax.grid(linestyle='dotted', linewidth=0.3)
ax.text(0.97, 0.97, 'b', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.legend(loc='upper left',ncol=2, fontsize=7)

## Return time  
ax = fig.add_subplot(gs[2, 0])

ax.boxplot([metrics[1][:,1,4]], positions=[1], showcaps=False, boxprops = dict(linewidth=1, color=color1), medianprops = dict(linewidth = 1, color=color1), whiskerprops = dict(linewidth=1, color=color1), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,2,4]], positions=[2], showcaps=False, boxprops = dict(linewidth=1, color=color1, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color1, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color1, linestyle='dashed'), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,4,4]], positions=[3], showcaps=False, boxprops = dict(linewidth=1, color=color2), medianprops = dict(linewidth = 1, color=color2), whiskerprops = dict(linewidth=1, color=color2), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,5,4]], positions=[4], showcaps=False, boxprops = dict(linewidth=1, color=color2, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color2, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color2, linestyle='dashed'), showfliers=False, widths=0.3)

ax.axhline(100, color='black', linestyle='dashed', linewidth=1)
ax.set_ylabel(r"$\hat{r}(Z > \hat{z}_{p=1\%})$ [y]")
ax.set_xticklabels([r"$\mathcal{M}_1$",r"$\mathcal{M}_2$",r"$\mathcal{M}_1^B$",r"$\mathcal{M}_2^B$"])
ax.grid(linestyle='dotted', linewidth=0.3)
ax.text(0.97, 0.97, 'c', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.set_ylim(0,1000)

## cdf_norm_Linf 
ax = fig.add_subplot(gs[2, 1])

ax.boxplot([metrics[1][:,1,2]], positions=[1], showcaps=False, boxprops = dict(linewidth=1, color=color1), medianprops = dict(linewidth = 1, color=color1), whiskerprops = dict(linewidth=1, color=color1), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,2,2]], positions=[2], showcaps=False, boxprops = dict(linewidth=1, color=color1, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color1, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color1, linestyle='dashed'), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,4,2]], positions=[3], showcaps=False, boxprops = dict(linewidth=1, color=color2), medianprops = dict(linewidth = 1, color=color2), whiskerprops = dict(linewidth=1, color=color2), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,5,2]], positions=[4], showcaps=False, boxprops = dict(linewidth=1, color=color2, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color2, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color2, linestyle='dashed'), showfliers=False, widths=0.3)

ax.set_ylabel(r"$\|\hat{F} - F_e\|_\infty$")
ax.set_xticklabels([r"$\mathcal{M}_1$",r"$\mathcal{M}_2$",r"$\mathcal{M}_1^B$",r"$\mathcal{M}_2^B$"])
ax.grid(linestyle='dotted', linewidth=0.3)
ax.text(0.97, 0.97, 'd', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.set_ylim(bottom=0)

## cdf_norm_L2
ax = fig.add_subplot(gs[2, 2])

ax.boxplot([metrics[1][:,1,3]], positions=[1], showcaps=False, boxprops = dict(linewidth=1, color=color1), medianprops = dict(linewidth = 1, color=color1), whiskerprops = dict(linewidth=1, color=color1), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,2,3]], positions=[2], showcaps=False, boxprops = dict(linewidth=1, color=color1, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color1, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color1, linestyle='dashed'), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,4,3]], positions=[3], showcaps=False, boxprops = dict(linewidth=1, color=color2), medianprops = dict(linewidth = 1, color=color2), whiskerprops = dict(linewidth=1, color=color2), showfliers=False, widths=0.3)
ax.boxplot([metrics[1][:,5,3]], positions=[4], showcaps=False, boxprops = dict(linewidth=1, color=color2, linestyle='dashed'), medianprops = dict(linewidth = 1, color=color2, linestyle='dashed'), whiskerprops = dict(linewidth=1, color=color2, linestyle='dashed'), showfliers=False, widths=0.3)

ax.set_ylabel(r"$\|\hat{F} - F_e\|_2$")
ax.set_xticklabels([r"$\mathcal{M}_1$",r"$\mathcal{M}_2$",r"$\mathcal{M}_1^B$",r"$\mathcal{M}_2^B$"])
ax.grid(linestyle='dotted', linewidth=0.3)
ax.text(0.97, 0.97, 'e', fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='right')
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.show()

In [ ]:
## Maps absolute values upper bound - Median estimate

year_limit = 2014
n_years = 70

## Load data
upper_bound_median = xr.load_dataarray(path + "upper_bound_2000_median_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_supp.nc")
upper_bound_std = xr.load_dataarray(path + "upper_bound_2000_std_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_supp.nc")

lon = upper_bound_median['lon']
lat = upper_bound_median['lat'] 

## Plot    
xticks = [0, 50, 100, 150, -50, -100, -150]
yticks = [35, 45, 55, 65,75]
xticklabels = ['0°', '50°E', '100°E', '150°E', '50°W', '100°W', '150°W']
yticklabels = ['35°N', '45°N', '55°N', '65°N', '75°N']
letters_tab = [['a','c','e'],['b','d','f']]


fig = plt.figure(figsize=(7,5.5), dpi=1200)
gs = fig.add_gridspec(3,2)

for idm,m in enumerate(["m1","m2"]):

    ## Median upper bound
    cmap = mpl.cm.get_cmap('Reds', 8)
    tab = upper_bound_median.sel( model = m ).values

    ax = fig.add_subplot(gs[0,idm], projection=ccrs.PlateCarree())
    cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', vmin=20, vmax=60, aspect=2)

    ax.coastlines('50m', color='0', linewidth=.3)   
    ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
    ax.set_yticks(yticks, crs=ccrs.PlateCarree())
    ax.set_xticks(xticks, crs=ccrs.PlateCarree())
    ax.set_xticklabels("")
    if idm == 0:
        ax.set_yticklabels(yticklabels, fontsize=8)
    else:
        ax.set_yticklabels("")
    ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
    ax.text(-0.06, 1.2, letters_tab[idm][0], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
    if m =="m1":
        ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1$)", fontsize=10)
    elif m =="m2":
        ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2$)", fontsize=10)

    cbar_ax = fig.add_axes([0.285, 0.75, 0.48, 0.02])
    cbar = fig.colorbar(cp, label="Median statistical upper bound [°C]", spacing='proportional', cax=cbar_ax, orientation='horizontal', extend='both')


    ## Median upper bound difference
    cmap = mpl.cm.get_cmap('Reds', 8)

    tab = upper_bound_median.sel( model = m + '_B' ).values 
    ax = fig.add_subplot(gs[1,idm], projection=ccrs.PlateCarree())
    cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', vmin=20, vmax=60, aspect=2)

    ax.coastlines('50m', color='0', linewidth=.3)   
    ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
    ax.set_yticks(yticks, crs=ccrs.PlateCarree())
    ax.set_xticks(xticks, crs=ccrs.PlateCarree())
    ax.set_xticklabels("")
    if idm == 0:
        ax.set_yticklabels(yticklabels, fontsize=8)
    else:
        ax.set_yticklabels("")
    ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
    ax.text(-0.06, 1.2, letters_tab[idm][1], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
    if m =="m1":
        ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1^B$)", fontsize=10)
    elif m =="m2":
        ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2^B$)", fontsize=10)

    cbar_ax = fig.add_axes([0.285, 0.43, 0.48, 0.02])
    cbar = fig.colorbar(cp, label="Median physical upper bound [°C]", spacing='proportional', cax=cbar_ax, orientation='horizontal', extend='both')


    ## Spread difference
    cmap = mpl.cm.get_cmap('RdBu_r', 11)
    norm = mpl.colors.BoundaryNorm([-13.5,-10.5,-7.5,-4.5,-1.5,1.5,4.5,7.5,10.5,13.5], cmap.N)

    tab = upper_bound_median.sel( model = m + '_B' ).values - upper_bound_median.sel( model = m ).values
    ax = fig.add_subplot(gs[2,idm], projection=ccrs.PlateCarree())
    cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

    ax.coastlines('50m', color='0', linewidth=.3)   
    ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
    ax.set_yticks(yticks, crs=ccrs.PlateCarree())
    ax.set_xticks(xticks, crs=ccrs.PlateCarree())
    ax.set_xticklabels(xticklabels, fontsize=8)
    if idm == 0:
        ax.set_yticklabels(yticklabels, fontsize=8)
    else:
        ax.set_yticklabels("")
    ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
    ax.text(-0.06, 1.2, letters_tab[idm][2], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

    cbar_ax = fig.add_axes([0.285, 0.08, 0.48, 0.02])
    cbar = fig.colorbar(cp, label="Difference [°C]", cax=cbar_ax, orientation='horizontal', extend='both')
    cbar.set_ticks([-12,-9,-6,-3,0,3,6,9,12])
    cbar.set_ticklabels([-12,-9,-6,-3,0,3,6,9,12])

fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.1 , top = 1 , hspace = 0.25 , wspace = 0.08 )
plt.show()



In [ ]:
## Maps absolute values upper bound - Std estimate

year_limit = 2014
n_years = 70

## Load data
upper_bound_median = xr.load_dataarray(path + "upper_bound_2000_median_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_supp.nc")
upper_bound_std = xr.load_dataarray(path + "upper_bound_2000_std_yl" + str(year_limit) + "_ny" + str(n_years) + "_v3_supp.nc")

lon = upper_bound_median['lon']
lat = upper_bound_median['lat'] 

## Plot    
xticks = [0, 50, 100, 150, -50, -100, -150]
yticks = [35, 45, 55, 65,75]
xticklabels = ['0°', '50°E', '100°E', '150°E', '50°W', '100°W', '150°W']
yticklabels = ['35°N', '45°N', '55°N', '65°N', '75°N']
letters_tab = [['a','c','e'],['b','d','f']]


fig = plt.figure(figsize=(7,5.5), dpi=1200)
gs = fig.add_gridspec(3,2)

for idm,m in enumerate(["m1","m2"]):

    ## Median upper bound
    cmap = mpl.cm.get_cmap('Reds', 8)    
    norm = mpl.colors.BoundaryNorm([0,0.5,1,1.5,2,5,10,15,20], cmap.N)

    tab = upper_bound_std.sel( model = m ).values

    ax = fig.add_subplot(gs[0,idm], projection=ccrs.PlateCarree())
    cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm = norm, aspect=2)

    ax.coastlines('50m', color='0', linewidth=.3)   
    ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
    ax.set_yticks(yticks, crs=ccrs.PlateCarree())
    ax.set_xticks(xticks, crs=ccrs.PlateCarree())
    ax.set_xticklabels("")
    if idm == 0:
        ax.set_yticklabels(yticklabels, fontsize=8)
    else:
        ax.set_yticklabels("")
    ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
    ax.text(-0.06, 1.2, letters_tab[idm][0], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
    if m =="m1":
        ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1$)", fontsize=10)
    elif m =="m2":
        ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2$)", fontsize=10)

    cbar_ax = fig.add_axes([0.285, 0.75, 0.48, 0.02])
    cbar = fig.colorbar(cp, label="Standard deviation in statistical upper bound [°C]", cax=cbar_ax, orientation='horizontal', extend='max')


    ## Median upper bound difference
    cmap = mpl.cm.get_cmap('Reds', 8)
    norm = mpl.colors.BoundaryNorm([0,0.5,1,1.5,2,5,10,15,20], cmap.N)

    tab = upper_bound_std.sel( model = m + '_B' ).values 
    ax = fig.add_subplot(gs[1,idm], projection=ccrs.PlateCarree())
    cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

    ax.coastlines('50m', color='0', linewidth=.3)   
    ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
    ax.set_yticks(yticks, crs=ccrs.PlateCarree())
    ax.set_xticks(xticks, crs=ccrs.PlateCarree())
    ax.set_xticklabels("")
    if idm == 0:
        ax.set_yticklabels(yticklabels, fontsize=8)
    else:
        ax.set_yticklabels("")
    ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
    ax.text(-0.06, 1.2, letters_tab[idm][1], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')
    if m =="m1":
        ax.set_title(r"Non-stationary location parameter ($\mathcal{M}_1^B$)", fontsize=10)
    elif m =="m2":
        ax.set_title(r"Non-stationary location and scale parameters ($\mathcal{M}_2^B$)", fontsize=10)

    cbar_ax = fig.add_axes([0.285, 0.43, 0.48, 0.02])
    cbar = fig.colorbar(cp, label="Standard deviation in physical upper bound [°C]", cax=cbar_ax, orientation='horizontal', extend='max')


    ## Spread difference
    cmap = mpl.cm.get_cmap('RdBu_r', 11)
    norm = mpl.colors.BoundaryNorm([-20,-15,-10,-5,-2,2,5,10,15,20], cmap.N)

    tab = upper_bound_std.sel( model = m + '_B' ).values - upper_bound_std.sel( model = m ).values
    ax = fig.add_subplot(gs[2,idm], projection=ccrs.PlateCarree())
    cp = ax.imshow(tab, cmap=cmap,extent=[lon[0],lon[-1],lat[0],lat[-1]], origin='lower', norm=norm, aspect=2)

    ax.coastlines('50m', color='0', linewidth=.3)   
    ax.gridlines(draw_labels = False, xlocs=xticks, ylocs=yticks, color='.7', alpha=0.4, linewidth=.3)
    ax.set_yticks(yticks, crs=ccrs.PlateCarree())
    ax.set_xticks(xticks, crs=ccrs.PlateCarree())
    ax.set_xticklabels(xticklabels, fontsize=8)
    if idm == 0:
        ax.set_yticklabels(yticklabels, fontsize=8)
    else:
        ax.set_yticklabels("")
    ax.set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=ccrs.PlateCarree())
    ax.text(-0.06, 1.2, letters_tab[idm][2], fontsize=15, transform=ax.transAxes, fontweight='bold', va='top', ha='left')

    cbar_ax = fig.add_axes([0.285, 0.08, 0.48, 0.02])
    cbar = fig.colorbar(cp, label="Difference [°C]", cax=cbar_ax, orientation='horizontal', extend='both')

fig.subplots_adjust( left = 0.06 , right = 0.99 , bottom = 0.1 , top = 1 , hspace = 0.25 , wspace = 0.08 )
plt.show()